In [1]:
# =========================================================
# ⚙ Legal-SBERT Extractive Summarization (Kaggle Version)
# • Using nlpaueb/legal-bert (works with current transformers)
# =========================================================

import os, pandas as pd, nltk, time, json, torch, math
from tqdm.notebook import tqdm
from transformers import AutoModel, AutoTokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.kl import KLSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.edmundson import EdmundsonSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

# ---- Config ----
INPUT_PATH  = "/kaggle/input/train-data/train_data.csv"  
OUTPUT_CSV  = "/kaggle/working/extracted_summaries.csv"
LOG_FILE    = "/kaggle/working/timing_log.json"
NUM_SENTENCES = 15
MAX_TOKENS = 512
CHUNK_SIZE = 200
USE_GPU = True

# ---- Custom cosine similarity ----
def cos_sim(a, b):
    """Computes the cosine similarity between two tensors."""
    if len(a.shape) == 1:
        a = a.unsqueeze(0)
    if len(b.shape) == 1:
        b = b.unsqueeze(0)
    
    a_norm = torch.nn.functional.normalize(a, p=2, dim=1)
    b_norm = torch.nn.functional.normalize(b, p=2, dim=1)
    return torch.mm(a_norm, b_norm.transpose(0, 1))

# ---- Load dataset ----
def load_dataset(input_path):
    """Load dataset from file or directory"""
    if os.path.isfile(input_path):
        print(f"✓ Loading CSV file: {input_path}")
        return pd.read_csv(input_path)
    elif os.path.isdir(input_path):
        import glob
        csv_files = glob.glob(os.path.join(input_path, "*.csv"))
        if not csv_files:
            raise FileNotFoundError(f"No CSV files found in directory: {input_path}")
        main_csv = csv_files[0]
        print(f"✓ Loading CSV file: {main_csv} from directory")
        if len(csv_files) > 1:
            print(f"  Found multiple CSV files: {csv_files}")
            print(f"  Using: {main_csv}")
        return pd.read_csv(main_csv)
    else:
        raise FileNotFoundError(f"Path not found: {input_path}")

# ---- Load Legal-BERT (compatible with current transformers) ----
MODEL_NAME = "law-ai/InLegalBERT"
print(f"🔧 Loading model: {MODEL_NAME}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
base_model = AutoModel.from_pretrained(MODEL_NAME)
print(f"✅ {MODEL_NAME} loaded successfully!")

# Mean pooling (SBERT style)
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# SBERT-style wrapper
class SimpleSBERT:
    def __init__(self, model, tokenizer, device):
        self.model = model.to(device)
        self.tokenizer = tokenizer
        self.device = device
    
    def encode(self, sentences, convert_to_tensor=True, batch_size=32):
        """Encode sentences with batching for efficiency"""
        if isinstance(sentences, str):
            sentences = [sentences]
        
        all_embeddings = []
        
        # Process in batches to avoid memory issues
        for i in range(0, len(sentences), batch_size):
            batch = sentences[i:i + batch_size]
            encoded_input = self.tokenizer(
                batch, 
                padding=True, 
                truncation=True,
                return_tensors='pt', 
                max_length=512
            )
            encoded_input = {k: v.to(self.device) for k, v in encoded_input.items()}
            
            with torch.no_grad():
                model_output = self.model(**encoded_input)
            
            embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
            all_embeddings.append(embeddings)
        
        # Concatenate all batches
        final_embeddings = torch.cat(all_embeddings, dim=0)
        
        return final_embeddings if convert_to_tensor else final_embeddings.cpu().numpy()

device = "cuda" if USE_GPU and torch.cuda.is_available() else "cpu"
print(f"🖥️  Using device: {device}")
model = SimpleSBERT(base_model, tokenizer, device)
print("✓ Legal-SBERT model ready.")

# ---- Helpers ----
LANG = "english"
STEMMER = Stemmer(LANG)
STOPWORDS = get_stop_words(LANG)

def detect_text_column(df):
    for c in ['text','document','source','content','judgment','body','full_text','doc','case_text']:
        if c in df.columns:
            return c
    return max(df.columns, key=lambda x: df[x].astype(str).str.len().mean())

def sent_tokenize(text):
    from nltk import sent_tokenize as st
    return [s.strip() for s in st(text) if s.strip()]

def limit_tokens(text, max_tokens=512):
    toks = text.split()
    return " ".join(toks[:max_tokens])

def sumy_summarize(text, method, n=15):
    parser = PlaintextParser.from_string(text, Tokenizer(LANG))
    if method == "TextRank": summarizer = TextRankSummarizer(STEMMER)
    elif method == "LexRank": summarizer = LexRankSummarizer(STEMMER)
    elif method == "Luhn": summarizer = LuhnSummarizer(STEMMER)
    elif method == "Kl": summarizer = KLSummarizer(STEMMER)
    elif method == "LSA": summarizer = LsaSummarizer(STEMMER)
    elif method == "Edmundson": 
        summarizer = EdmundsonSummarizer(STEMMER)
        summarizer.bonus_words = ['important', 'significant', 'notable', 'crucial']
        summarizer.stigma_words = ['however', 'although', 'nevertheless']
        summarizer.null_words = STOPWORDS
    else: raise ValueError(f"Unknown method: {method}")
    summarizer.stop_words = STOPWORDS
    sents = summarizer(parser.document, n)
    return " ".join(str(s) for s in sents)

# Legal-SBERT centroid
def sbert_centroid(text, n=15):
    sents = sent_tokenize(text)
    if not sents: return ""
    if len(sents) <= n:
        return " ".join(sents)
    
    # Encode sentences
    emb = model.encode(sents, convert_to_tensor=True)
    
    # Compute centroid
    centroid = emb.mean(dim=0, keepdim=True)
    
    # Compute similarities
    sims = cos_sim(centroid, emb)[0]
    
    # Get top-k most similar to centroid
    topk = min(n, len(sents))
    idxs = sims.topk(k=topk).indices.tolist()
    idxs.sort()
    
    return " ".join([sents[i] for i in idxs])

# ---- Resume check ----
existing_rows = []
processed_ids = set()
if os.path.exists(OUTPUT_CSV):
    print(f"📂 Resuming from existing file: {OUTPUT_CSV}")
    try:
        existing_df = pd.read_csv(OUTPUT_CSV)
        existing_rows = existing_df.to_dict(orient="records")
        processed_ids = set([r["doc_id"] for r in existing_rows])
        print(f"✓ Already processed: {len(processed_ids)} docs")
    except Exception as e:
        print(f"⚠️  Error reading existing file: {e}. Starting fresh.")

# ---- Load dataset ----
df = load_dataset(INPUT_PATH)
# df = df.head(5)
text_col = detect_text_column(df)
print(f"📄 Detected text column: '{text_col}', total docs={len(df)}")

methods = ["TextRank","LexRank","LSA","Edmundson","Luhn","KL-Sum","LegalSBERT-centroid"]
timing_stats = {m: [] for m in methods}

# ---- Single-process summarization ----
def process_document(doc_id, text):
    text = str(text)
    if not text.strip(): 
        return []
    
    text = " ".join(text.split()[:4000])  # Limit to 4000 words
    
    out = []
    for m in methods:
        start = time.time()
        try:
            if m == "LegalSBERT-centroid": 
                s = sbert_centroid(text, NUM_SENTENCES)
            else:
                method_name = "Kl" if m == "KL-Sum" else m
                s = sumy_summarize(text, method_name, NUM_SENTENCES)
        except Exception as e:
            # Fallback: take first n sentences
            sents = sent_tokenize(text)
            s = " ".join(sents[:NUM_SENTENCES])
            if len(sents) > NUM_SENTENCES:
                print(f"⚠️  Method {m} failed for doc {doc_id}: {str(e)[:100]}...")
        
        duration = time.time() - start
        timing_stats[m].append(duration)
        out.append((doc_id, m, limit_tokens(s, MAX_TOKENS)))
    
    return out

# ---- ETA Dashboard ----
rows = [tuple(r.values()) for r in existing_rows] if existing_rows else []
to_process = [(idx, df.loc[idx, text_col]) for idx in df.index if idx not in processed_ids]
total_docs = len(to_process)
start_time = time.time()

print(f"\n🚀 Processing remaining {total_docs} docs using single process...")

if total_docs > 0:
    # Use tqdm for progress tracking
    for i, (doc_id, text) in enumerate(tqdm(to_process, total=total_docs), start=1):
        result = process_document(doc_id, text)
        rows.extend(result)
        
        if i % 50 == 0:
            elapsed = time.time() - start_time
            docs_per_min = i / (elapsed / 60) if elapsed > 0 else 0
            remaining = (total_docs - i) / docs_per_min if docs_per_min > 0 else 0
            print(f"  📊 {i}/{total_docs} docs | {docs_per_min:.1f} docs/min | ETA: {remaining:.1f} min")

        if i % CHUNK_SIZE == 0:
            pd.DataFrame(rows, columns=["doc_id","method","extractive_doc"]).to_csv(OUTPUT_CSV, index=False)
            with open(LOG_FILE, "w") as f: 
                json.dump(timing_stats, f, indent=2)
            print(f"💾 Checkpoint saved ({i} docs).")

# ---- Save final output ----
pd.DataFrame(rows, columns=["doc_id","method","extractive_doc"]).to_csv(OUTPUT_CSV, index=False)

# Calculate final timing statistics
final_timing = {}
for method, times in timing_stats.items():
    if times:
        final_timing[method] = {
            "total_docs": len(times),
            "avg_time": sum(times) / len(times),
            "total_time": sum(times),
            "min_time": min(times),
            "max_time": max(times)
        }

with open(LOG_FILE, "w") as f: 
    json.dump(final_timing, f, indent=2)

elapsed_total = time.time() - start_time
print(f"\n✅ All done in {elapsed_total/60:.1f} min")
print(f"📊 Processed {len(rows)} total summaries")
print(f"💾 Output: {OUTPUT_CSV}")
print(f"📈 Timing log: {LOG_FILE}")

# Display timing summary
print("\n=== ⏱️  Timing Summary ===")
for method, stats in final_timing.items():
    print(f"{method:25s}: {stats['avg_time']:.2f}s avg per doc")

ModuleNotFoundError: No module named 'sumy'

🔧 Loading model: law-ai/InLegalBERT
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
✅ law-ai/InLegalBERT loaded successfully!
🖥️  Using device: cuda
✓ Legal-SBERT model ready.
📂 Resuming from existing file: /kaggle/working/extracted_summaries.csv
✓ Already processed: 5 docs
✓ Loading CSV file: /kaggle/input/train-data/train_data.csv
📄 Detected text column: 'judgment', total docs=5532

🚀 Processing remaining 5527 docs using single process...
100%
 5527/5527 [4:04:55<00:00,  4.88s/it]
  📊 50/5527 docs | 19.7 docs/min | ETA: 278.5 min
  📊 100/5527 docs | 19.9 docs/min | ETA: 272.4 min
  📊 150/5527 docs | 21.2 docs/min | ETA: 253.4 min
  📊 200/5527 docs | 21.5 docs/min | ETA: 247.6 min
💾 Checkpoint saved (200 docs).
  📊 250/5527 docs | 21.7 docs/min | ETA: 243.2 min
  📊 300/5527 docs | 22.3 docs/min | ETA: 234.2 min
  📊 350/5527 docs | 22.0 docs/min | ETA: 235.3 min
  📊 400/5527 docs | 22.5 docs/min | ETA: 228.3 min
💾 Checkpoint saved (400 docs).
  📊 450/5527 docs | 22.9 docs/min | ETA: 221.9 min
  📊 500/5527 docs | 22.9 docs/min | ETA: 220.0 min
  📊 550/5527 docs | 22.8 docs/min | ETA: 218.1 min
  📊 600/5527 docs | 22.8 docs/min | ETA: 216.1 min
💾 Checkpoint saved (600 docs).
  📊 650/5527 docs | 22.6 docs/min | ETA: 215.7 min
  📊 700/5527 docs | 22.6 docs/min | ETA: 213.7 min
  📊 750/5527 docs | 22.7 docs/min | ETA: 210.1 min
  📊 800/5527 docs | 22.7 docs/min | ETA: 208.5 min
💾 Checkpoint saved (800 docs).
  📊 850/5527 docs | 22.6 docs/min | ETA: 206.8 min
  📊 900/5527 docs | 22.7 docs/min | ETA: 203.9 min
  📊 950/5527 docs | 22.7 docs/min | ETA: 201.4 min
  📊 1000/5527 docs | 22.8 docs/min | ETA: 198.7 min
💾 Checkpoint saved (1000 docs).
  📊 1050/5527 docs | 22.8 docs/min | ETA: 196.2 min
  📊 1100/5527 docs | 23.0 docs/min | ETA: 192.6 min
  📊 1150/5527 docs | 23.0 docs/min | ETA: 190.3 min
  📊 1200/5527 docs | 22.9 docs/min | ETA: 188.7 min
💾 Checkpoint saved (1200 docs).
  📊 1250/5527 docs | 23.0 docs/min | ETA: 185.7 min
  📊 1300/5527 docs | 23.0 docs/min | ETA: 183.6 min
  📊 1350/5527 docs | 23.0 docs/min | ETA: 181.4 min
  📊 1400/5527 docs | 22.9 docs/min | ETA: 180.2 min
💾 Checkpoint saved (1400 docs).
  📊 1450/5527 docs | 22.8 docs/min | ETA: 178.5 min
  📊 1500/5527 docs | 22.8 docs/min | ETA: 176.8 min
  📊 1550/5527 docs | 22.8 docs/min | ETA: 174.5 min
  📊 1600/5527 docs | 22.8 docs/min | ETA: 172.2 min
💾 Checkpoint saved (1600 docs).
  📊 1650/5527 docs | 22.9 docs/min | ETA: 169.2 min
  📊 1700/5527 docs | 22.9 docs/min | ETA: 167.2 min
  📊 1750/5527 docs | 22.9 docs/min | ETA: 164.9 min
  📊 1800/5527 docs | 22.8 docs/min | ETA: 163.1 min
💾 Checkpoint saved (1800 docs).
  📊 1850/5527 docs | 22.8 docs/min | ETA: 161.6 min
  📊 1900/5527 docs | 22.7 docs/min | ETA: 159.7 min
  📊 1950/5527 docs | 22.7 docs/min | ETA: 157.4 min
  📊 2000/5527 docs | 22.8 docs/min | ETA: 154.9 min
💾 Checkpoint saved (2000 docs).
  📊 2050/5527 docs | 22.7 docs/min | ETA: 153.0 min
  📊 2100/5527 docs | 22.7 docs/min | ETA: 151.1 min
  📊 2150/5527 docs | 22.8 docs/min | ETA: 148.4 min
  📊 2200/5527 docs | 22.8 docs/min | ETA: 146.2 min
💾 Checkpoint saved (2200 docs).
  📊 2250/5527 docs | 22.7 docs/min | ETA: 144.1 min
  📊 2300/5527 docs | 22.7 docs/min | ETA: 142.1 min
  📊 2350/5527 docs | 22.7 docs/min | ETA: 139.6 min
  📊 2400/5527 docs | 22.8 docs/min | ETA: 137.4 min
💾 Checkpoint saved (2400 docs).
  📊 2450/5527 docs | 22.8 docs/min | ETA: 135.1 min
  📊 2500/5527 docs | 22.7 docs/min | ETA: 133.2 min
  📊 2550/5527 docs | 22.7 docs/min | ETA: 131.0 min
  📊 2600/5527 docs | 22.7 docs/min | ETA: 128.7 min
💾 Checkpoint saved (2600 docs).
  📊 2650/5527 docs | 22.8 docs/min | ETA: 126.3 min
  📊 2700/5527 docs | 22.8 docs/min | ETA: 124.2 min
  📊 2750/5527 docs | 22.7 docs/min | ETA: 122.2 min
  📊 2800/5527 docs | 22.7 docs/min | ETA: 120.2 min
💾 Checkpoint saved (2800 docs).
  📊 2850/5527 docs | 22.7 docs/min | ETA: 117.9 min
  📊 2900/5527 docs | 22.7 docs/min | ETA: 115.6 min
  📊 2950/5527 docs | 22.7 docs/min | ETA: 113.7 min
  📊 3000/5527 docs | 22.7 docs/min | ETA: 111.6 min
💾 Checkpoint saved (3000 docs).
  📊 3050/5527 docs | 22.7 docs/min | ETA: 109.3 min
  📊 3100/5527 docs | 22.7 docs/min | ETA: 106.9 min
  📊 3150/5527 docs | 22.7 docs/min | ETA: 104.7 min
  📊 3200/5527 docs | 22.7 docs/min | ETA: 102.6 min
💾 Checkpoint saved (3200 docs).
  📊 3250/5527 docs | 22.7 docs/min | ETA: 100.5 min
  📊 3300/5527 docs | 22.6 docs/min | ETA: 98.4 min
  📊 3350/5527 docs | 22.6 docs/min | ETA: 96.3 min
  📊 3400/5527 docs | 22.6 docs/min | ETA: 94.1 min
💾 Checkpoint saved (3400 docs).
  📊 3450/5527 docs | 22.6 docs/min | ETA: 91.8 min
  📊 3500/5527 docs | 22.6 docs/min | ETA: 89.7 min
  📊 3550/5527 docs | 22.6 docs/min | ETA: 87.5 min
  📊 3600/5527 docs | 22.6 docs/min | ETA: 85.2 min
💾 Checkpoint saved (3600 docs).
  📊 3650/5527 docs | 22.6 docs/min | ETA: 83.0 min
  📊 3700/5527 docs | 22.6 docs/min | ETA: 80.8 min
  📊 3750/5527 docs | 22.6 docs/min | ETA: 78.7 min
  📊 3800/5527 docs | 22.6 docs/min | ETA: 76.4 min
💾 Checkpoint saved (3800 docs).
  📊 3850/5527 docs | 22.6 docs/min | ETA: 74.1 min
  📊 3900/5527 docs | 22.7 docs/min | ETA: 71.8 min
  📊 3950/5527 docs | 22.6 docs/min | ETA: 69.7 min
  📊 4000/5527 docs | 22.7 docs/min | ETA: 67.3 min
💾 Checkpoint saved (4000 docs).
  📊 4050/5527 docs | 22.7 docs/min | ETA: 65.2 min
  📊 4100/5527 docs | 22.6 docs/min | ETA: 63.0 min
  📊 4150/5527 docs | 22.7 docs/min | ETA: 60.8 min
  📊 4200/5527 docs | 22.7 docs/min | ETA: 58.6 min
💾 Checkpoint saved (4200 docs).
  📊 4250/5527 docs | 22.6 docs/min | ETA: 56.4 min
  📊 4300/5527 docs | 22.6 docs/min | ETA: 54.2 min
  📊 4350/5527 docs | 22.6 docs/min | ETA: 52.0 min
  📊 4400/5527 docs | 22.6 docs/min | ETA: 49.8 min
💾 Checkpoint saved (4400 docs).
  📊 4450/5527 docs | 22.7 docs/min | ETA: 47.5 min
  📊 4500/5527 docs | 22.6 docs/min | ETA: 45.5 min
  📊 4550/5527 docs | 22.6 docs/min | ETA: 43.3 min
  📊 4600/5527 docs | 22.6 docs/min | ETA: 41.0 min
💾 Checkpoint saved (4600 docs).
  📊 4650/5527 docs | 22.6 docs/min | ETA: 38.9 min
  📊 4700/5527 docs | 22.6 docs/min | ETA: 36.6 min
  📊 4750/5527 docs | 22.6 docs/min | ETA: 34.4 min
  📊 4800/5527 docs | 22.6 docs/min | ETA: 32.2 min
💾 Checkpoint saved (4800 docs).
  📊 4850/5527 docs | 22.6 docs/min | ETA: 30.0 min
  📊 4900/5527 docs | 22.6 docs/min | ETA: 27.8 min
  📊 4950/5527 docs | 22.6 docs/min | ETA: 25.5 min
  📊 5000/5527 docs | 22.6 docs/min | ETA: 23.3 min
💾 Checkpoint saved (5000 docs).
  📊 5050/5527 docs | 22.6 docs/min | ETA: 21.1 min
  📊 5100/5527 docs | 22.6 docs/min | ETA: 18.9 min
  📊 5150/5527 docs | 22.6 docs/min | ETA: 16.7 min
  📊 5200/5527 docs | 22.6 docs/min | ETA: 14.5 min
💾 Checkpoint saved (5200 docs).
  📊 5250/5527 docs | 22.6 docs/min | ETA: 12.2 min
  📊 5300/5527 docs | 22.6 docs/min | ETA: 10.0 min
  📊 5350/5527 docs | 22.6 docs/min | ETA: 7.8 min
  📊 5400/5527 docs | 22.6 docs/min | ETA: 5.6 min
💾 Checkpoint saved (5400 docs).
  📊 5450/5527 docs | 22.6 docs/min | ETA: 3.4 min
  📊 5500/5527 docs | 22.6 docs/min | ETA: 1.2 min

✅ All done in 245.0 min
📊 Processed 38724 total summaries
💾 Output: /kaggle/working/extracted_summaries.csv
📈 Timing log: /kaggle/working/timing_log.json

=== ⏱️  Timing Summary ===
TextRank                 : 0.13s avg per doc
LexRank                  : 0.15s avg per doc
LSA                      : 0.11s avg per doc
Edmundson                : 0.10s avg per doc
Luhn                     : 0.08s avg per doc
KL-Sum                   : 1.49s avg per doc
LegalSBERT-centroid      : 0.59s avg per doc

In [ ]:
pip install -U transformers==4.44.2

##DAY 2

In [ ]:
!pip install --quiet sumy sentence-transformers nltk tqdm pandas
!pip install -U transformers==4.44.2

In [ ]:
# =========================================================
# ⚙ Legal-SBERT Extractive Summarization (Kaggle Version)
# • Using nlpaueb/legal-bert (works with current transformers)
# =========================================================

import os, pandas as pd, nltk, time, json, torch, math
from tqdm.notebook import tqdm
from transformers import AutoModel, AutoTokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.kl import KLSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.edmundson import EdmundsonSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

# ---- Config ----
INPUT_PATH  = "/kaggle/input/train-data/train_data.csv"  
# OUTPUT_CSV  = "/kaggle/input/extracted-summaries/extracted_summaries.csv"
LOG_FILE    = "/kaggle/working/timing_log.json"
NUM_SENTENCES = 15
MAX_TOKENS = 512
CHUNK_SIZE = 200
USE_GPU = True

import os, shutil
import pandas as pd

# Input (read-only) and output (writable) paths
INPUT_CSV = "/kaggle/input/extracted-summaries/extracted_summaries.csv"
OUTPUT_CSV = "/kaggle/working/extracted_summaries.csv"

# Copy old checkpoint into working folder if it’s not already there
if os.path.exists(INPUT_CSV) and not os.path.exists(OUTPUT_CSV):
    shutil.copy(INPUT_CSV, OUTPUT_CSV)
    print("✅ Copied old checkpoint to working directory.")
else:
    print("✅ Using existing working copy.")

# Load previous results (if any)
existing_df = pd.read_csv(OUTPUT_CSV)
print(f"📂 Found {len(existing_df)} previously processed docs.")


# ---- Custom cosine similarity ----
def cos_sim(a, b):
    """Computes the cosine similarity between two tensors."""
    if len(a.shape) == 1:
        a = a.unsqueeze(0)
    if len(b.shape) == 1:
        b = b.unsqueeze(0)
    
    a_norm = torch.nn.functional.normalize(a, p=2, dim=1)
    b_norm = torch.nn.functional.normalize(b, p=2, dim=1)
    return torch.mm(a_norm, b_norm.transpose(0, 1))

# ---- Load dataset ----
def load_dataset(input_path):
    """Load dataset from file or directory"""
    if os.path.isfile(input_path):
        print(f"✓ Loading CSV file: {input_path}")
        return pd.read_csv(input_path)
    elif os.path.isdir(input_path):
        import glob
        csv_files = glob.glob(os.path.join(input_path, "*.csv"))
        if not csv_files:
            raise FileNotFoundError(f"No CSV files found in directory: {input_path}")
        main_csv = csv_files[0]
        print(f"✓ Loading CSV file: {main_csv} from directory")
        if len(csv_files) > 1:
            print(f"  Found multiple CSV files: {csv_files}")
            print(f"  Using: {main_csv}")
        return pd.read_csv(main_csv)
    else:
        raise FileNotFoundError(f"Path not found: {input_path}")

# ---- Load Legal-BERT (compatible with current transformers) ----
MODEL_NAME = "law-ai/InLegalBERT"
print(f"🔧 Loading model: {MODEL_NAME}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
base_model = AutoModel.from_pretrained(MODEL_NAME)
print(f"✅ {MODEL_NAME} loaded successfully!")

# Mean pooling (SBERT style)
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# SBERT-style wrapper
class SimpleSBERT:
    def __init__(self, model, tokenizer, device):
        self.model = model.to(device)
        self.tokenizer = tokenizer
        self.device = device
    
    def encode(self, sentences, convert_to_tensor=True, batch_size=32):
        """Encode sentences with batching for efficiency"""
        if isinstance(sentences, str):
            sentences = [sentences]
        
        all_embeddings = []
        
        # Process in batches to avoid memory issues
        for i in range(0, len(sentences), batch_size):
            batch = sentences[i:i + batch_size]
            encoded_input = self.tokenizer(
                batch, 
                padding=True, 
                truncation=True,
                return_tensors='pt', 
                max_length=512
            )
            encoded_input = {k: v.to(self.device) for k, v in encoded_input.items()}
            
            with torch.no_grad():
                model_output = self.model(**encoded_input)
            
            embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
            all_embeddings.append(embeddings)
        
        # Concatenate all batches
        final_embeddings = torch.cat(all_embeddings, dim=0)
        
        return final_embeddings if convert_to_tensor else final_embeddings.cpu().numpy()

device = "cuda" if USE_GPU and torch.cuda.is_available() else "cpu"
print(f"🖥️  Using device: {device}")
model = SimpleSBERT(base_model, tokenizer, device)
print("✓ Legal-SBERT model ready.")

# ---- Helpers ----
LANG = "english"
STEMMER = Stemmer(LANG)
STOPWORDS = get_stop_words(LANG)

def detect_text_column(df):
    for c in ['text','document','source','content','judgment','body','full_text','doc','case_text']:
        if c in df.columns:
            return c
    return max(df.columns, key=lambda x: df[x].astype(str).str.len().mean())

def sent_tokenize(text):
    from nltk import sent_tokenize as st
    return [s.strip() for s in st(text) if s.strip()]

def limit_tokens(text, max_tokens=512):
    toks = text.split()
    return " ".join(toks[:max_tokens])

def sumy_summarize(text, method, n=15):
    parser = PlaintextParser.from_string(text, Tokenizer(LANG))
    if method == "TextRank": summarizer = TextRankSummarizer(STEMMER)
    elif method == "LexRank": summarizer = LexRankSummarizer(STEMMER)
    elif method == "Luhn": summarizer = LuhnSummarizer(STEMMER)
    elif method == "Kl": summarizer = KLSummarizer(STEMMER)
    elif method == "LSA": summarizer = LsaSummarizer(STEMMER)
    elif method == "Edmundson": 
        summarizer = EdmundsonSummarizer(STEMMER)
        summarizer.bonus_words = ['important', 'significant', 'notable', 'crucial']
        summarizer.stigma_words = ['however', 'although', 'nevertheless']
        summarizer.null_words = STOPWORDS
    else: raise ValueError(f"Unknown method: {method}")
    summarizer.stop_words = STOPWORDS
    sents = summarizer(parser.document, n)
    return " ".join(str(s) for s in sents)

# Legal-SBERT centroid
def sbert_centroid(text, n=15):
    sents = sent_tokenize(text)
    if not sents: return ""
    if len(sents) <= n:
        return " ".join(sents)
    
    # Encode sentences
    emb = model.encode(sents, convert_to_tensor=True)
    
    # Compute centroid
    centroid = emb.mean(dim=0, keepdim=True)
    
    # Compute similarities
    sims = cos_sim(centroid, emb)[0]
    
    # Get top-k most similar to centroid
    topk = min(n, len(sents))
    idxs = sims.topk(k=topk).indices.tolist()
    idxs.sort()
    
    return " ".join([sents[i] for i in idxs])

# ---- Resume check ----
existing_rows = []
processed_ids = set()
if os.path.exists(OUTPUT_CSV):
    print(f"📂 Resuming from existing file: {OUTPUT_CSV}")
    try:
        existing_df = pd.read_csv(OUTPUT_CSV)
        existing_rows = existing_df.to_dict(orient="records")
        processed_ids = set([r["doc_id"] for r in existing_rows])
        print(f"✓ Already processed: {len(processed_ids)} docs")
    except Exception as e:
        print(f"⚠️  Error reading existing file: {e}. Starting fresh.")

# ---- Load dataset ----
df = load_dataset(INPUT_PATH)
# df = df.head(5)
text_col = detect_text_column(df)
print(f"📄 Detected text column: '{text_col}', total docs={len(df)}")

methods = ["TextRank","LexRank","LSA","Edmundson","Luhn","KL-Sum","LegalSBERT-centroid"]
timing_stats = {m: [] for m in methods}

# ---- Single-process summarization ----
def process_document(doc_id, text):
    text = str(text)
    if not text.strip(): 
        return []
    
    text = " ".join(text.split()[:4000])  # Limit to 4000 words
    
    out = []
    for m in methods:
        start = time.time()
        try:
            if m == "LegalSBERT-centroid": 
                s = sbert_centroid(text, NUM_SENTENCES)
            else:
                method_name = "Kl" if m == "KL-Sum" else m
                s = sumy_summarize(text, method_name, NUM_SENTENCES)
        except Exception as e:
            # Fallback: take first n sentences
            sents = sent_tokenize(text)
            s = " ".join(sents[:NUM_SENTENCES])
            if len(sents) > NUM_SENTENCES:
                print(f"⚠️  Method {m} failed for doc {doc_id}: {str(e)[:100]}...")
        
        duration = time.time() - start
        timing_stats[m].append(duration)
        out.append((doc_id, m, limit_tokens(s, MAX_TOKENS)))
    
    return out

# ---- ETA Dashboard ----
rows = [tuple(r.values()) for r in existing_rows] if existing_rows else []
to_process = [(idx, df.loc[idx, text_col]) for idx in df.index if idx not in processed_ids]
total_docs = len(to_process)
start_time = time.time()

print(f"\n🚀 Processing remaining {total_docs} docs using single process...")

if total_docs > 0:
    # Use tqdm for progress tracking
    for i, (doc_id, text) in enumerate(tqdm(to_process, total=total_docs), start=1):
        result = process_document(doc_id, text)
        rows.extend(result)
        
        if i % 50 == 0:
            elapsed = time.time() - start_time
            docs_per_min = i / (elapsed / 60) if elapsed > 0 else 0
            remaining = (total_docs - i) / docs_per_min if docs_per_min > 0 else 0
            print(f"  📊 {i}/{total_docs} docs | {docs_per_min:.1f} docs/min | ETA: {remaining:.1f} min")

        if i % CHUNK_SIZE == 0:
            pd.DataFrame(rows, columns=["doc_id","method","extractive_doc"]).to_csv(OUTPUT_CSV, index=False)
            with open(LOG_FILE, "w") as f: 
                json.dump(timing_stats, f, indent=2)
            print(f"💾 Checkpoint saved ({i} docs).")

# ---- Save final output ----
pd.DataFrame(rows, columns=["doc_id","method","extractive_doc"]).to_csv(OUTPUT_CSV, index=False)

# Calculate final timing statistics
final_timing = {}
for method, times in timing_stats.items():
    if times:
        final_timing[method] = {
            "total_docs": len(times),
            "avg_time": sum(times) / len(times),
            "total_time": sum(times),
            "min_time": min(times),
            "max_time": max(times)
        }

with open(LOG_FILE, "w") as f: 
    json.dump(final_timing, f, indent=2)

elapsed_total = time.time() - start_time
print(f"\n✅ All done in {elapsed_total/60:.1f} min")
print(f"📊 Processed {len(rows)} total summaries")
print(f"💾 Output: {OUTPUT_CSV}")
print(f"📈 Timing log: {LOG_FILE}")

# Display timing summary
print("\n=== ⏱️  Timing Summary ===")
for method, stats in final_timing.items():
    print(f"{method:25s}: {stats['avg_time']:.2f}s avg per doc")

The above extractive summaries are not up to the mark

In [ ]:
# =========================================================
# ✅ ENHANCED Legal Extractive Summarization WITH GROUND TRUTH EVALUATION
# • Uses your ground truth summaries for proper evaluation
# • ROUGE and BERTScore metrics
# • Paper-compliant methods
# =========================================================

import os, pandas as pd, nltk, time, json, torch, re
from tqdm.notebook import tqdm
from transformers import AutoModel, AutoTokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.kl import KLSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.sum_basic import SumBasicSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

# Install evaluation packages
try:
    from rouge_score import rouge_scorer
    print("✅ ROUGE installed")
except:
    print("📦 Installing rouge-score...")
    os.system("pip install rouge-score")
    from rouge_score import rouge_scorer

try:
    from bert_score import BERTScorer
    print("✅ BERTScore installed")
except:
    print("📦 Installing bert-score...")
    os.system("pip install bert-score")
    from bert_score import BERTScorer

# ---- Config ----
INPUT_PATH  = "/kaggle/input/train-data/train_data.csv"  
OUTPUT_CSV  = "/kaggle/working/extracted_summaries_with_eval.csv"
EVAL_CSV    = "/kaggle/working/comprehensive_evaluation.csv"
LOG_FILE    = "/kaggle/working/timing_log.json"
MAX_TOKENS = 512
CHUNK_SIZE = 200
USE_GPU = True
TEST_MODE = True
TEST_SIZE = 5

# ---- Legal Pattern Detection ----
CONCLUSION_PATTERNS = re.compile(
    r'\b(held|hold that|we hold|we find|concluded?|ruling|final order|'
    r'appeals? (?:allowed|dismissed)|petition (?:allowed|dismissed)|'
    r'consequently|therefore|thus|accordingly|in view of|'
    r'we (?:allow|dismiss|set aside|uphold|reverse|affirm))\b',
    re.IGNORECASE
)

# ---- Initialize Evaluators ----
rouge_scorer_obj = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True)
bertscorer = BERTScorer(lang="en", model_type="microsoft/deberta-xlarge-mnli")

# ---- Evaluation Functions ----
def calculate_rouge(predicted, reference):
    """Calculate ROUGE scores between predicted and reference summaries"""
    scores = rouge_scorer_obj.score(reference, predicted)
    return {
        'rouge1_f1': scores['rouge1'].fmeasure,
        'rouge2_f1': scores['rouge2'].fmeasure,
        'rougeL_f1': scores['rougeL'].fmeasure,
        'rougeLsum_f1': scores['rougeLsum'].fmeasure,
        'rouge1_precision': scores['rouge1'].precision,
        'rouge1_recall': scores['rouge1'].recall,
    }

def calculate_bertscore(predicted, reference):
    """Calculate BERTScore between predicted and reference summaries"""
    P, R, F1 = bertscorer.score([predicted], [reference])
    return {
        'bertscore_precision': P.item(),
        'bertscore_recall': R.item(), 
        'bertscore_f1': F1.item()
    }

def validate_legal_summary(summary):
    """Check if summary contains legal reasoning indicators"""
    legal_indicators = ['court', 'judge', 'appeal', 'case', 'law', 'legal', 
                       'ruling', 'decision', 'therefore', 'thus', 'accordingly',
                       'held', 'conclusion', 'finding']
    summary_lower = summary.lower()
    return sum(1 for indicator in legal_indicators if indicator in summary_lower) >= 3

# ---- Cosine Similarity ----
def cos_sim(a, b):
    if len(a.shape) == 1:
        a = a.unsqueeze(0)
    if len(b.shape) == 1:
        b = b.unsqueeze(0)
    a_norm = torch.nn.functional.normalize(a, p=2, dim=1)
    b_norm = torch.nn.functional.normalize(b, p=2, dim=1)
    return torch.mm(a_norm, b_norm.transpose(0, 1))

# ---- Load Dataset ----
def load_dataset(input_path):
    if os.path.isfile(input_path):
        print(f"✓ Loading CSV: {input_path}")
        df = pd.read_csv(input_path)
        print(f"📊 Dataset shape: {df.shape}")
        print(f"📋 Columns: {df.columns.tolist()}")
        return df
    else:
        raise FileNotFoundError(f"Path not found: {input_path}")

# ---- Load Legal-BERT ----
MODEL_NAME = "nlpaueb/legal-bert-base-uncased"
print(f"🔧 Loading: {MODEL_NAME}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
base_model = AutoModel.from_pretrained(MODEL_NAME)
print(f"✅ Model loaded!")

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

class SimpleSBERT:
    def __init__(self, model, tokenizer, device):
        self.model = model.to(device)
        self.tokenizer = tokenizer
        self.device = device
    
    def encode(self, sentences, convert_to_tensor=True, batch_size=32):
        if isinstance(sentences, str):
            sentences = [sentences]
        all_embeddings = []
        for i in range(0, len(sentences), batch_size):
            batch = sentences[i:i + batch_size]
            encoded_input = self.tokenizer(
                batch, padding=True, truncation=True,
                return_tensors='pt', max_length=512
            )
            encoded_input = {k: v.to(self.device) for k, v in encoded_input.items()}
            with torch.no_grad():
                model_output = self.model(**encoded_input)
            embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
            all_embeddings.append(embeddings)
        final_embeddings = torch.cat(all_embeddings, dim=0)
        return final_embeddings if convert_to_tensor else final_embeddings.cpu().numpy()

device = "cuda" if USE_GPU and torch.cuda.is_available() else "cpu"
print(f"🖥️  Device: {device}")
model = SimpleSBERT(base_model, tokenizer, device)
print("✓ SBERT ready")

# ---- Helpers ----
LANG = "english"
STEMMER = Stemmer(LANG)
STOPWORDS = get_stop_words(LANG)

def detect_columns(df):
    """Detect text and summary columns"""
    text_col = None
    summary_col = None
    
    # Common column names
    text_candidates = ['text','document','source','content','judgment','body','full_text','doc','case_text']
    summary_candidates = ['summary', 'ground_truth', 'reference', 'abstract', 'gist']
    
    for c in text_candidates:
        if c in df.columns:
            text_col = c
            break
    if not text_col:
        text_col = max(df.columns, key=lambda x: df[x].astype(str).str.len().mean())
    
    for c in summary_candidates:
        if c in df.columns:
            summary_col = c
            break
    if not summary_col:
        # Assume second column is summary if not found
        summary_col = df.columns[1] if len(df.columns) > 1 else None
    
    return text_col, summary_col

def sent_tokenize(text):
    from nltk import sent_tokenize as st
    return [s.strip() for s in st(text) if s.strip()]

def limit_tokens(text, max_tokens=512):
    toks = text.split()
    return " ".join(toks[:max_tokens])

def preprocess_legal_text(text, max_words=10000):
    """Preserve beginning AND ending of legal documents"""
    words = text.split()
    if len(words) <= max_words:
        return text
    first_chunk = int(max_words * 0.3)
    last_chunk = max_words - first_chunk
    preserved_text = " ".join(words[:first_chunk]) + " " + " ".join(words[-last_chunk:])
    return preserved_text

# ---- Summarization Methods ----
def sumy_summarize(text, method, max_tokens=512):
    parser = PlaintextParser.from_string(text, Tokenizer(LANG))
    
    if method == "TextRank": summarizer = TextRankSummarizer(STEMMER)
    elif method == "LexRank": summarizer = LexRankSummarizer(STEMMER)
    elif method == "LSA": summarizer = LsaSummarizer(STEMMER)
    elif method == "KL-Sum": summarizer = KLSummarizer(STEMMER)
    elif method == "Luhn": summarizer = LuhnSummarizer(STEMMER)
    elif method == "SumBasic": summarizer = SumBasicSummarizer(STEMMER)
    else: raise ValueError(f"Unknown: {method}")
    
    summarizer.stop_words = STOPWORDS
    num_sentences = min(30, len(sent_tokenize(text)) // 4)
    num_sentences = max(15, num_sentences)
    sents = summarizer(parser.document, num_sentences)
    return limit_tokens(" ".join(str(s) for s in sents), max_tokens)

def reduction_summarize(text, max_tokens=512):
    sents = sent_tokenize(text)
    if not sents: return ""
    n = len(sents)
    selected_indices = set()
    selected_indices.update(range(min(3, n)))
    selected_indices.update(range(max(0, n-8), n))
    for i, sent in enumerate(sents):
        if CONCLUSION_PATTERNS.search(sent):
            selected_indices.add(i)
    selected_indices = sorted(selected_indices)[:25]
    summary = " ".join([sents[i] for i in selected_indices if i < n])
    return limit_tokens(summary, max_tokens)

def legal_sbert_weighted(text, max_tokens=512):
    sents = sent_tokenize(text)
    if not sents: return ""
    n = len(sents)
    weights = torch.ones(n)
    for i, sent in enumerate(sents):
        if CONCLUSION_PATTERNS.search(sent): weights[i] *= 3.0
        if i > n * 0.75: weights[i] *= 2.5
        elif i < n * 0.1: weights[i] *= 1.5
    emb = model.encode(sents, convert_to_tensor=True)
    weighted_centroid = (emb * weights.unsqueeze(1).to(emb.device)).sum(dim=0) / weights.sum()
    weighted_centroid = weighted_centroid.unsqueeze(0)
    sims = cos_sim(weighted_centroid, emb)[0]
    topk = min(20, n)
    idxs = sims.topk(k=topk).indices.tolist()
    idxs.sort()
    summary = " ".join([sents[i] for i in idxs])
    return limit_tokens(summary, max_tokens)

# ---- Methods List ----
methods = ["TextRank", "LexRank", "LSA", "KL-Sum", "SumBasic", "Reduction", "LegalSBERT-Weighted"]
timing_stats = {m: [] for m in methods}

# ---- Process Document ----
def process_document(doc_id, text):
    text = str(text)
    if not text.strip(): return []
    text = preprocess_legal_text(text, max_words=10000)
    
    out = []
    for m in methods:
        start = time.time()
        try:
            if m == "LegalSBERT-Weighted": s = legal_sbert_weighted(text, MAX_TOKENS)
            elif m == "Reduction": s = reduction_summarize(text, MAX_TOKENS)
            else: s = sumy_summarize(text, m, MAX_TOKENS)
        except Exception as e:
            sents = sent_tokenize(text)
            n = len(sents)
            strategic = list(range(min(5, n))) + list(range(max(0, n-10), n))
            s = " ".join([sents[i] for i in strategic if i < n])
            s = limit_tokens(s, MAX_TOKENS)
        
        duration = time.time() - start
        timing_stats[m].append(duration)
        out.append((doc_id, m, limit_tokens(s, MAX_TOKENS)))
    
    return out

# ---- Enhanced Evaluation with Ground Truth ----
def evaluate_with_ground_truth(extracted_df, original_df, text_col, summary_col):
    """Comprehensive evaluation using ground truth summaries"""
    evaluation_results = []
    
    print("🔍 Evaluating with ground truth summaries...")
    
    for doc_id in extracted_df['doc_id'].unique():
        # Get ground truth summary
        if doc_id in original_df.index:
            ground_truth = str(original_df.loc[doc_id, summary_col])
        else:
            continue
            
        doc_summaries = extracted_df[extracted_df['doc_id'] == doc_id]
        original_text = original_df.loc[doc_id, text_col]
        
        for _, row in doc_summaries.iterrows():
            method = row['method']
            predicted_summary = row['extractive_doc']
            
            # Calculate all metrics
            metrics = {
                'doc_id': doc_id,
                'method': method,
                'summary_length': len(predicted_summary.split()),
                'legal_score': validate_legal_summary(predicted_summary),
                'has_conclusion': bool(CONCLUSION_PATTERNS.search(predicted_summary)),
            }
            
            # ROUGE scores
            rouge_scores = calculate_rouge(predicted_summary, ground_truth)
            metrics.update(rouge_scores)
            
            # BERTScore
            bert_scores = calculate_bertscore(predicted_summary, ground_truth)
            metrics.update(bert_scores)
            
            evaluation_results.append(metrics)
    
    eval_df = pd.DataFrame(evaluation_results)
    
    # Print comprehensive results
    print_comprehensive_results(eval_df)
    
    return eval_df

def print_comprehensive_results(eval_df):
    """Print detailed evaluation results"""
    print("\n" + "="*80)
    print("🎯 COMPREHENSIVE EVALUATION WITH GROUND TRUTH SUMMARIES")
    print("="*80)
    
    # Group by method
    summary = eval_df.groupby('method').agg({
        'rouge1_f1': ['mean', 'std'],
        'rouge2_f1': ['mean', 'std'],
        'rougeL_f1': ['mean', 'std'], 
        'bertscore_f1': ['mean', 'std'],
        'has_conclusion': 'mean',
        'legal_score': 'mean',
        'summary_length': 'mean',
        'doc_id': 'count'
    }).round(4)
    
    # Flatten column names
    summary.columns = ['_'.join(col).strip() for col in summary.columns.values]
    print(summary)
    
    # Identify best methods
    print(f"\n🏆 TOP PERFORMING METHODS:")
    print("   By ROUGE-1 F1:")
    best_rouge1 = eval_df.groupby('method')['rouge1_f1'].mean().nlargest(3)
    for method, score in best_rouge1.items():
        print(f"     {method}: {score:.3f}")
    
    print("   By BERTScore F1:")
    best_bertscore = eval_df.groupby('method')['bertscore_f1'].mean().nlargest(3)
    for method, score in best_bertscore.items():
        print(f"     {method}: {score:.3f}")
    
    print("   By Conclusion Inclusion:")
    best_conclusion = eval_df.groupby('method')['has_conclusion'].mean().nlargest(3)
    for method, score in best_conclusion.items():
        print(f"     {method}: {score:.1%}")

# ---- Main Execution ----
if __name__ == "__main__":
    # Load dataset
    df = load_dataset(INPUT_PATH)
    text_col, summary_col = detect_columns(df)
    print(f"📄 Text column: '{text_col}'")
    print(f"📝 Summary column: '{summary_col}'")
    
    # TEST MODE
    if TEST_MODE:
        df = df.head(TEST_SIZE)
        print(f"🧪 TEST MODE: Processing first {TEST_SIZE} documents")
    
    # Process documents
    rows = []
    to_process = [(idx, df.loc[idx, text_col]) for idx in df.index]
    total_docs = len(to_process)
    start_time = time.time()

    print(f"\n🚀 Processing {total_docs} docs...")
    
    for i, (doc_id, text) in enumerate(tqdm(to_process, total=total_docs), start=1):
        result = process_document(doc_id, text)
        rows.extend(result)
        
        if i % 10 == 0:  # More frequent checkpoints for test mode
            elapsed = time.time() - start_time
            docs_per_min = i / (elapsed / 60) if elapsed > 0 else 0
            print(f"  📊 {i}/{total_docs} | {docs_per_min:.1f} docs/min")

    # Save extracted summaries
    extracted_df = pd.DataFrame(rows, columns=["doc_id","method","extractive_doc"])
    extracted_df.to_csv(OUTPUT_CSV, index=False)
    
    # Run comprehensive evaluation with ground truth
    print("\n🔍 Running comprehensive evaluation with ground truth...")
    evaluation_df = evaluate_with_ground_truth(extracted_df, df, text_col, summary_col)
    evaluation_df.to_csv(EVAL_CSV, index=False)
    
    # Final stats
    elapsed_total = time.time() - start_time
    print(f"\n✅ Completed in {elapsed_total/60:.1f} min")
    print(f"📊 Generated {len(rows)} summaries")
    print(f"💾 Output: {OUTPUT_CSV}")
    print(f"📈 Evaluation: {EVAL_CSV}")
    
    # Recommendation
    if TEST_MODE:
        print(f"\n🎯 REVIEW THE RESULTS ABOVE!")
        print("   If ROUGE/BERTScore look good, set TEST_MODE = False for full run")
        print("   Look for methods with high ROUGE-1 (>0.4) and good conclusion inclusion")

In [ ]:
# =========================================================
# ✅ ENHANCED Legal Extractive Summarization WITH GROUND TRUTH EVALUATION
# • Uses your ground truth summaries for proper evaluation
# • ROUGE and BERTScore metrics
# • Paper-compliant methods
# =========================================================

import os, pandas as pd, nltk, time, json, torch, re
from tqdm.notebook import tqdm
from transformers import AutoModel, AutoTokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.kl import KLSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.sum_basic import SumBasicSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

# Install evaluation packages
try:
    from rouge_score import rouge_scorer
    print("✅ ROUGE installed")
except:
    print("📦 Installing rouge-score...")
    os.system("pip install rouge-score")
    from rouge_score import rouge_scorer

try:
    from bert_score import BERTScorer
    print("✅ BERTScore installed")
except:
    print("📦 Installing bert-score...")
    os.system("pip install bert-score")
    from bert_score import BERTScorer

# ---- Config ----
INPUT_PATH  = "/kaggle/input/train-data/train_data.csv"  
OUTPUT_CSV  = "/kaggle/working/extracted_summaries_with_eval.csv"
EVAL_CSV    = "/kaggle/working/comprehensive_evaluation.csv"
LOG_FILE    = "/kaggle/working/timing_log.json"
MAX_TOKENS = 512
CHUNK_SIZE = 200
USE_GPU = True
TEST_MODE = True
TEST_SIZE = 5

# ---- Legal Pattern Detection ----
CONCLUSION_PATTERNS = re.compile(
    r'\b(held|hold that|we hold|we find|concluded?|ruling|final order|'
    r'appeals? (?:allowed|dismissed)|petition (?:allowed|dismissed)|'
    r'consequently|therefore|thus|accordingly|in view of|'
    r'we (?:allow|dismiss|set aside|uphold|reverse|affirm))\b',
    re.IGNORECASE
)

# ---- Initialize Evaluators ----
rouge_scorer_obj = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True)
bertscorer = BERTScorer(lang="en", model_type="microsoft/deberta-xlarge-mnli")

# ---- Evaluation Functions ----
def calculate_rouge(predicted, reference):
    """Calculate ROUGE scores between predicted and reference summaries"""
    scores = rouge_scorer_obj.score(reference, predicted)
    return {
        'rouge1_f1': scores['rouge1'].fmeasure,
        'rouge2_f1': scores['rouge2'].fmeasure,
        'rougeL_f1': scores['rougeL'].fmeasure,
        'rougeLsum_f1': scores['rougeLsum'].fmeasure,
        'rouge1_precision': scores['rouge1'].precision,
        'rouge1_recall': scores['rouge1'].recall,
    }

def calculate_bertscore(predicted, reference):
    """Calculate BERTScore between predicted and reference summaries"""
    P, R, F1 = bertscorer.score([predicted], [reference])
    return {
        'bertscore_precision': P.item(),
        'bertscore_recall': R.item(), 
        'bertscore_f1': F1.item()
    }

def validate_legal_summary(summary):
    """Check if summary contains legal reasoning indicators"""
    legal_indicators = ['court', 'judge', 'appeal', 'case', 'law', 'legal', 
                       'ruling', 'decision', 'therefore', 'thus', 'accordingly',
                       'held', 'conclusion', 'finding']
    summary_lower = summary.lower()
    return sum(1 for indicator in legal_indicators if indicator in summary_lower) >= 3

# ---- Cosine Similarity ----
def cos_sim(a, b):
    if len(a.shape) == 1:
        a = a.unsqueeze(0)
    if len(b.shape) == 1:
        b = b.unsqueeze(0)
    a_norm = torch.nn.functional.normalize(a, p=2, dim=1)
    b_norm = torch.nn.functional.normalize(b, p=2, dim=1)
    return torch.mm(a_norm, b_norm.transpose(0, 1))

# ---- Load Dataset ----
def load_dataset(input_path):
    if os.path.isfile(input_path):
        print(f"✓ Loading CSV: {input_path}")
        df = pd.read_csv(input_path)
        print(f"📊 Dataset shape: {df.shape}")
        print(f"📋 Columns: {df.columns.tolist()}")
        return df
    else:
        raise FileNotFoundError(f"Path not found: {input_path}")

# ---- Load Legal-BERT ----
MODEL_NAME = "law-ai/InLegalBERT"
print(f"🔧 Loading: {MODEL_NAME}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
base_model = AutoModel.from_pretrained(MODEL_NAME)
print(f"✅ Model loaded!")

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

class SimpleSBERT:
    def __init__(self, model, tokenizer, device):
        self.model = model.to(device)
        self.tokenizer = tokenizer
        self.device = device
    
    def encode(self, sentences, convert_to_tensor=True, batch_size=32):
        if isinstance(sentences, str):
            sentences = [sentences]
        all_embeddings = []
        for i in range(0, len(sentences), batch_size):
            batch = sentences[i:i + batch_size]
            encoded_input = self.tokenizer(
                batch, padding=True, truncation=True,
                return_tensors='pt', max_length=512
            )
            encoded_input = {k: v.to(self.device) for k, v in encoded_input.items()}
            with torch.no_grad():
                model_output = self.model(**encoded_input)
            embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
            all_embeddings.append(embeddings)
        final_embeddings = torch.cat(all_embeddings, dim=0)
        return final_embeddings if convert_to_tensor else final_embeddings.cpu().numpy()

device = "cuda" if USE_GPU and torch.cuda.is_available() else "cpu"
print(f"🖥️  Device: {device}")
model = SimpleSBERT(base_model, tokenizer, device)
print("✓ SBERT ready")

# ---- Helpers ----
LANG = "english"
STEMMER = Stemmer(LANG)
STOPWORDS = get_stop_words(LANG)

def detect_columns(df):
    """Detect text and summary columns"""
    text_col = None
    summary_col = None
    
    # Common column names
    text_candidates = ['text','document','source','content','judgment','body','full_text','doc','case_text']
    summary_candidates = ['summary', 'ground_truth', 'reference', 'abstract', 'gist']
    
    for c in text_candidates:
        if c in df.columns:
            text_col = c
            break
    if not text_col:
        text_col = max(df.columns, key=lambda x: df[x].astype(str).str.len().mean())
    
    for c in summary_candidates:
        if c in df.columns:
            summary_col = c
            break
    if not summary_col:
        # Assume second column is summary if not found
        summary_col = df.columns[1] if len(df.columns) > 1 else None
    
    return text_col, summary_col

def sent_tokenize(text):
    from nltk import sent_tokenize as st
    return [s.strip() for s in st(text) if s.strip()]

def limit_tokens(text, max_tokens=512):
    toks = text.split()
    return " ".join(toks[:max_tokens])

def preprocess_legal_text(text, max_words=10000):
    """Preserve beginning AND ending of legal documents"""
    words = text.split()
    if len(words) <= max_words:
        return text
    first_chunk = int(max_words * 0.3)
    last_chunk = max_words - first_chunk
    preserved_text = " ".join(words[:first_chunk]) + " " + " ".join(words[-last_chunk:])
    return preserved_text

# ---- Summarization Methods ----
def sumy_summarize(text, method, max_tokens=512):
    parser = PlaintextParser.from_string(text, Tokenizer(LANG))
    
    if method == "TextRank": summarizer = TextRankSummarizer(STEMMER)
    elif method == "LexRank": summarizer = LexRankSummarizer(STEMMER)
    elif method == "LSA": summarizer = LsaSummarizer(STEMMER)
    elif method == "KL-Sum": summarizer = KLSummarizer(STEMMER)
    elif method == "Luhn": summarizer = LuhnSummarizer(STEMMER)
    elif method == "SumBasic": summarizer = SumBasicSummarizer(STEMMER)
    else: raise ValueError(f"Unknown: {method}")
    
    summarizer.stop_words = STOPWORDS
    num_sentences = min(30, len(sent_tokenize(text)) // 4)
    num_sentences = max(15, num_sentences)
    sents = summarizer(parser.document, num_sentences)
    return limit_tokens(" ".join(str(s) for s in sents), max_tokens)

def reduction_summarize(text, max_tokens=512):
    sents = sent_tokenize(text)
    if not sents: return ""
    n = len(sents)
    selected_indices = set()
    selected_indices.update(range(min(3, n)))
    selected_indices.update(range(max(0, n-8), n))
    for i, sent in enumerate(sents):
        if CONCLUSION_PATTERNS.search(sent):
            selected_indices.add(i)
    selected_indices = sorted(selected_indices)[:25]
    summary = " ".join([sents[i] for i in selected_indices if i < n])
    return limit_tokens(summary, max_tokens)

def legal_sbert_weighted(text, max_tokens=512):
    sents = sent_tokenize(text)
    if not sents: return ""
    n = len(sents)
    weights = torch.ones(n)
    for i, sent in enumerate(sents):
        if CONCLUSION_PATTERNS.search(sent): weights[i] *= 3.0
        if i > n * 0.75: weights[i] *= 2.5
        elif i < n * 0.1: weights[i] *= 1.5
    emb = model.encode(sents, convert_to_tensor=True)
    weighted_centroid = (emb * weights.unsqueeze(1).to(emb.device)).sum(dim=0) / weights.sum()
    weighted_centroid = weighted_centroid.unsqueeze(0)
    sims = cos_sim(weighted_centroid, emb)[0]
    topk = min(20, n)
    idxs = sims.topk(k=topk).indices.tolist()
    idxs.sort()
    summary = " ".join([sents[i] for i in idxs])
    return limit_tokens(summary, max_tokens)

# ---- Methods List ----
methods = ["TextRank", "LexRank", "LSA", "KL-Sum", "SumBasic", "Reduction", "LegalSBERT-Weighted"]
timing_stats = {m: [] for m in methods}

# ---- Process Document ----
def process_document(doc_id, text):
    text = str(text)
    if not text.strip(): return []
    text = preprocess_legal_text(text, max_words=10000)
    
    out = []
    for m in methods:
        start = time.time()
        try:
            if m == "LegalSBERT-Weighted": s = legal_sbert_weighted(text, MAX_TOKENS)
            elif m == "Reduction": s = reduction_summarize(text, MAX_TOKENS)
            else: s = sumy_summarize(text, m, MAX_TOKENS)
        except Exception as e:
            sents = sent_tokenize(text)
            n = len(sents)
            strategic = list(range(min(5, n))) + list(range(max(0, n-10), n))
            s = " ".join([sents[i] for i in strategic if i < n])
            s = limit_tokens(s, MAX_TOKENS)
        
        duration = time.time() - start
        timing_stats[m].append(duration)
        out.append((doc_id, m, limit_tokens(s, MAX_TOKENS)))
    
    return out

# ---- Enhanced Evaluation with Ground Truth ----
def evaluate_with_ground_truth(extracted_df, original_df, text_col, summary_col):
    """Comprehensive evaluation using ground truth summaries"""
    evaluation_results = []
    
    print("🔍 Evaluating with ground truth summaries...")
    
    for doc_id in extracted_df['doc_id'].unique():
        # Get ground truth summary
        if doc_id in original_df.index:
            ground_truth = str(original_df.loc[doc_id, summary_col])
        else:
            continue
            
        doc_summaries = extracted_df[extracted_df['doc_id'] == doc_id]
        original_text = original_df.loc[doc_id, text_col]
        
        for _, row in doc_summaries.iterrows():
            method = row['method']
            predicted_summary = row['extractive_doc']
            
            # Calculate all metrics
            metrics = {
                'doc_id': doc_id,
                'method': method,
                'summary_length': len(predicted_summary.split()),
                'legal_score': validate_legal_summary(predicted_summary),
                'has_conclusion': bool(CONCLUSION_PATTERNS.search(predicted_summary)),
            }
            
            # ROUGE scores
            rouge_scores = calculate_rouge(predicted_summary, ground_truth)
            metrics.update(rouge_scores)
            
            # BERTScore
            bert_scores = calculate_bertscore(predicted_summary, ground_truth)
            metrics.update(bert_scores)
            
            evaluation_results.append(metrics)
    
    eval_df = pd.DataFrame(evaluation_results)
    
    # Print comprehensive results
    print_comprehensive_results(eval_df)
    
    return eval_df

def print_comprehensive_results(eval_df):
    """Print detailed evaluation results"""
    print("\n" + "="*80)
    print("🎯 COMPREHENSIVE EVALUATION WITH GROUND TRUTH SUMMARIES")
    print("="*80)
    
    # Group by method
    summary = eval_df.groupby('method').agg({
        'rouge1_f1': ['mean', 'std'],
        'rouge2_f1': ['mean', 'std'],
        'rougeL_f1': ['mean', 'std'], 
        'bertscore_f1': ['mean', 'std'],
        'has_conclusion': 'mean',
        'legal_score': 'mean',
        'summary_length': 'mean',
        'doc_id': 'count'
    }).round(4)
    
    # Flatten column names
    summary.columns = ['_'.join(col).strip() for col in summary.columns.values]
    print(summary)
    
    # Identify best methods
    print(f"\n🏆 TOP PERFORMING METHODS:")
    print("   By ROUGE-1 F1:")
    best_rouge1 = eval_df.groupby('method')['rouge1_f1'].mean().nlargest(3)
    for method, score in best_rouge1.items():
        print(f"     {method}: {score:.3f}")
    
    print("   By BERTScore F1:")
    best_bertscore = eval_df.groupby('method')['bertscore_f1'].mean().nlargest(3)
    for method, score in best_bertscore.items():
        print(f"     {method}: {score:.3f}")
    
    print("   By Conclusion Inclusion:")
    best_conclusion = eval_df.groupby('method')['has_conclusion'].mean().nlargest(3)
    for method, score in best_conclusion.items():
        print(f"     {method}: {score:.1%}")

# ---- Main Execution ----
if __name__ == "__main__":
    # Load dataset
    df = load_dataset(INPUT_PATH)
    text_col, summary_col = detect_columns(df)
    print(f"📄 Text column: '{text_col}'")
    print(f"📝 Summary column: '{summary_col}'")
    
    # TEST MODE
    if TEST_MODE:
        df = df.head(TEST_SIZE)
        print(f"🧪 TEST MODE: Processing first {TEST_SIZE} documents")
    
    # Process documents
    rows = []
    to_process = [(idx, df.loc[idx, text_col]) for idx in df.index]
    total_docs = len(to_process)
    start_time = time.time()

    print(f"\n🚀 Processing {total_docs} docs...")
    
    for i, (doc_id, text) in enumerate(tqdm(to_process, total=total_docs), start=1):
        result = process_document(doc_id, text)
        rows.extend(result)
        
        if i % 10 == 0:  # More frequent checkpoints for test mode
            elapsed = time.time() - start_time
            docs_per_min = i / (elapsed / 60) if elapsed > 0 else 0
            print(f"  📊 {i}/{total_docs} | {docs_per_min:.1f} docs/min")

    # Save extracted summaries
    extracted_df = pd.DataFrame(rows, columns=["doc_id","method","extractive_doc"])
    extracted_df.to_csv(OUTPUT_CSV, index=False)
    
    # Run comprehensive evaluation with ground truth
    print("\n🔍 Running comprehensive evaluation with ground truth...")
    evaluation_df = evaluate_with_ground_truth(extracted_df, df, text_col, summary_col)
    evaluation_df.to_csv(EVAL_CSV, index=False)
    
    # Final stats
    elapsed_total = time.time() - start_time
    print(f"\n✅ Completed in {elapsed_total/60:.1f} min")
    print(f"📊 Generated {len(rows)} summaries")
    print(f"💾 Output: {OUTPUT_CSV}")
    print(f"📈 Evaluation: {EVAL_CSV}")
    
    # Recommendation
    if TEST_MODE:
        print(f"\n🎯 REVIEW THE RESULTS ABOVE!")
        print("   If ROUGE/BERTScore look good, set TEST_MODE = False for full run")
        print("   Look for methods with high ROUGE-1 (>0.4) and good conclusion inclusion")

lastest and final

In [ ]:
!pip install --quiet sumy sentence-transformers nltk tqdm pandas
!pip install -U transformers==4.44.2

In [ ]:
# === RESUME CODE ===
import shutil
import os

# Copy your uploaded files to working directory
files_to_restore = [
    'checkpoint_state.json',
    'extracted_summaries_with_eval.csv'
]

for file in files_to_restore:
    if os.path.exists(f'/kaggle/input/legaldoc1/{file}'):
        shutil.copy(f'/kaggle/input/legaldoc1/{file}', f'/kaggle/working/{file}')
        print(f"✅ Restored: {file}")
    else:
        print(f"⚠️  Not found: {file}")

print("🎯 READY TO RESUME! Now run your main code...")

In [ ]:
# =========================================================
# ✅ ENHANCED Legal Extractive Summarization WITH CHECKPOINTS & MULTI-GPU
# =========================================================

import os, pandas as pd, nltk, time, json, torch, re
from tqdm.notebook import tqdm
from transformers import AutoModel, AutoTokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.kl import KLSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.sum_basic import SumBasicSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

# Install evaluation packages
try:
    from rouge_score import rouge_scorer
    print("✅ ROUGE installed")
except:
    print("📦 Installing rouge-score...")
    os.system("pip install rouge-score")
    from rouge_score import rouge_scorer

try:
    from bert_score import BERTScorer
    print("✅ BERTScore installed")
except:
    print("📦 Installing bert-score...")
    os.system("pip install bert-score")
    from bert_score import BERTScorer

# ---- Config ----
INPUT_PATH  = "/kaggle/input/train-data/train_data.csv"  
OUTPUT_CSV  = "/kaggle/working/extracted_summaries_with_eval.csv"
EVAL_CSV    = "/kaggle/working/comprehensive_evaluation.csv"
LOG_FILE    = "/kaggle/working/timing_log.json"
CHECKPOINT_FILE = "/kaggle/working/checkpoint_state.json"  # ✅ NEW: Checkpoint file
MAX_TOKENS = 512
CHUNK_SIZE = 50  # ✅ Reduced for more frequent checkpoints
USE_GPU = True
TEST_MODE = False  # ✅ CHANGED: Full run!
TEST_SIZE = 5
USE_MULTI_GPU = True  # ✅ NEW: Multi-GPU support

# ---- Legal Pattern Detection ----
CONCLUSION_PATTERNS = re.compile(
    r'\b(held|hold that|we hold|we find|concluded?|ruling|final order|'
    r'appeals? (?:allowed|dismissed)|petition (?:allowed|dismissed)|'
    r'consequently|therefore|thus|accordingly|in view of|'
    r'we (?:allow|dismiss|set aside|uphold|reverse|affirm))\b',
    re.IGNORECASE
)

# ---- Initialize Evaluators ----
rouge_scorer_obj = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True)
bertscorer = BERTScorer(lang="en", model_type="microsoft/deberta-xlarge-mnli")

# ---- Enhanced GPU Setup ----
def setup_multigpu():
    """Setup multi-GPU if available"""
    if USE_MULTI_GPU and torch.cuda.device_count() > 1:
        print(f"🎯 Using {torch.cuda.device_count()} GPUs!")
        return True
    else:
        print(f"🖥️  Using {torch.cuda.device_count()} GPU")
        return False

# ---- Checkpoint Management ----
def save_checkpoint(processed_ids, rows, timing_stats):
    """Save checkpoint state"""
    checkpoint_data = {
        'processed_ids': list(processed_ids),
        'total_rows': len(rows),
        'timestamp': time.time(),
        'timing_stats': timing_stats
    }
    with open(CHECKPOINT_FILE, 'w') as f:
        json.dump(checkpoint_data, f, indent=2)
    print(f"💾 Checkpoint saved: {len(processed_ids)} docs processed")

def load_checkpoint():
    """Load checkpoint state if exists"""
    if os.path.exists(CHECKPOINT_FILE):
        print("📂 Loading checkpoint...")
        with open(CHECKPOINT_FILE, 'r') as f:
            checkpoint_data = json.load(f)
        processed_ids = set(checkpoint_data['processed_ids'])
        print(f"✓ Resuming from checkpoint: {len(processed_ids)} docs already processed")
        return processed_ids, checkpoint_data.get('timing_stats', {})
    return set(), {}

# ---- Evaluation Functions ----
def calculate_rouge(predicted, reference):
    """Calculate ROUGE scores between predicted and reference summaries"""
    scores = rouge_scorer_obj.score(reference, predicted)
    return {
        'rouge1_f1': scores['rouge1'].fmeasure,
        'rouge2_f1': scores['rouge2'].fmeasure,
        'rougeL_f1': scores['rougeL'].fmeasure,
        'rougeLsum_f1': scores['rougeLsum'].fmeasure,
        'rouge1_precision': scores['rouge1'].precision,
        'rouge1_recall': scores['rouge1'].recall,
    }

def calculate_bertscore(predicted, reference):
    """Calculate BERTScore between predicted and reference summaries"""
    P, R, F1 = bertscorer.score([predicted], [reference])
    return {
        'bertscore_precision': P.item(),
        'bertscore_recall': R.item(), 
        'bertscore_f1': F1.item()
    }

def validate_legal_summary(summary):
    """Check if summary contains legal reasoning indicators"""
    legal_indicators = ['court', 'judge', 'appeal', 'case', 'law', 'legal', 
                       'ruling', 'decision', 'therefore', 'thus', 'accordingly',
                       'held', 'conclusion', 'finding']
    summary_lower = summary.lower()
    return sum(1 for indicator in legal_indicators if indicator in summary_lower) >= 3

# ---- Cosine Similarity ----
def cos_sim(a, b):
    if len(a.shape) == 1:
        a = a.unsqueeze(0)
    if len(b.shape) == 1:
        b = b.unsqueeze(0)
    a_norm = torch.nn.functional.normalize(a, p=2, dim=1)
    b_norm = torch.nn.functional.normalize(b, p=2, dim=1)
    return torch.mm(a_norm, b_norm.transpose(0, 1))

# ---- Load Dataset ----
def load_dataset(input_path):
    if os.path.isfile(input_path):
        print(f"✓ Loading CSV: {input_path}")
        df = pd.read_csv(input_path)
        print(f"📊 Dataset shape: {df.shape}")
        print(f"📋 Columns: {df.columns.tolist()}")
        return df
    else:
        raise FileNotFoundError(f"Path not found: {input_path}")

# ---- Enhanced Model Loading with Multi-GPU ----
MODEL_NAME = "law-ai/InLegalBERT"
print(f"🔧 Loading: {MODEL_NAME}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
base_model = AutoModel.from_pretrained(MODEL_NAME)

# ✅ MULTI-GPU SUPPORT
if USE_MULTI_GPU and torch.cuda.device_count() > 1:
    print(f"🚀 Using {torch.cuda.device_count()} GPUs with DataParallel")
    base_model = torch.nn.DataParallel(base_model)

print(f"✅ Model loaded!")

def mean_pooling(model_output, attention_mask):
    # Handle multi-GPU output
    if isinstance(model_output, tuple):
        token_embeddings = model_output[0]
    else:
        token_embeddings = model_output[0] if isinstance(model_output, tuple) else model_output.last_hidden_state
    
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

class SimpleSBERT:
    def __init__(self, model, tokenizer, device):
        self.model = model.to(device)
        self.tokenizer = tokenizer
        self.device = device
    
    def encode(self, sentences, convert_to_tensor=True, batch_size=32):
        if isinstance(sentences, str):
            sentences = [sentences]
        all_embeddings = []
        for i in range(0, len(sentences), batch_size):
            batch = sentences[i:i + batch_size]
            encoded_input = self.tokenizer(
                batch, padding=True, truncation=True,
                return_tensors='pt', max_length=512
            )
            encoded_input = {k: v.to(self.device) for k, v in encoded_input.items()}
            with torch.no_grad():
                model_output = self.model(**encoded_input)
            embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
            all_embeddings.append(embeddings)
        final_embeddings = torch.cat(all_embeddings, dim=0)
        return final_embeddings if convert_to_tensor else final_embeddings.cpu().numpy()

# Setup device
device = "cuda" if USE_GPU and torch.cuda.is_available() else "cpu"
print(f"🖥️  Device: {device}")
model = SimpleSBERT(base_model, tokenizer, device)
print("✓ SBERT ready")

# ---- Helpers ----
LANG = "english"
STEMMER = Stemmer(LANG)
STOPWORDS = get_stop_words(LANG)

def detect_columns(df):
    """Detect text and summary columns"""
    text_col = None
    summary_col = None
    
    # Common column names
    text_candidates = ['text','document','source','content','judgment','body','full_text','doc','case_text']
    summary_candidates = ['summary', 'ground_truth', 'reference', 'abstract', 'gist']
    
    for c in text_candidates:
        if c in df.columns:
            text_col = c
            break
    if not text_col:
        text_col = max(df.columns, key=lambda x: df[x].astype(str).str.len().mean())
    
    for c in summary_candidates:
        if c in df.columns:
            summary_col = c
            break
    if not summary_col:
        # Assume second column is summary if not found
        summary_col = df.columns[1] if len(df.columns) > 1 else None
    
    return text_col, summary_col

def sent_tokenize(text):
    from nltk import sent_tokenize as st
    return [s.strip() for s in st(text) if s.strip()]

def limit_tokens(text, max_tokens=512):
    toks = text.split()
    return " ".join(toks[:max_tokens])

def preprocess_legal_text(text, max_words=10000):
    """Preserve beginning AND ending of legal documents"""
    words = text.split()
    if len(words) <= max_words:
        return text
    first_chunk = int(max_words * 0.3)
    last_chunk = max_words - first_chunk
    preserved_text = " ".join(words[:first_chunk]) + " " + " ".join(words[-last_chunk:])
    return preserved_text

# ---- Summarization Methods ----
def sumy_summarize(text, method, max_tokens=512):
    parser = PlaintextParser.from_string(text, Tokenizer(LANG))
    
    if method == "TextRank": summarizer = TextRankSummarizer(STEMMER)
    elif method == "LexRank": summarizer = LexRankSummarizer(STEMMER)
    elif method == "LSA": summarizer = LsaSummarizer(STEMMER)
    elif method == "KL-Sum": summarizer = KLSummarizer(STEMMER)
    elif method == "Luhn": summarizer = LuhnSummarizer(STEMMER)
    elif method == "SumBasic": summarizer = SumBasicSummarizer(STEMMER)
    else: raise ValueError(f"Unknown: {method}")
    
    summarizer.stop_words = STOPWORDS
    num_sentences = min(30, len(sent_tokenize(text)) // 4)
    num_sentences = max(15, num_sentences)
    sents = summarizer(parser.document, num_sentences)
    return limit_tokens(" ".join(str(s) for s in sents), max_tokens)

def reduction_summarize(text, max_tokens=512):
    sents = sent_tokenize(text)
    if not sents: return ""
    n = len(sents)
    selected_indices = set()
    selected_indices.update(range(min(3, n)))
    selected_indices.update(range(max(0, n-8), n))
    for i, sent in enumerate(sents):
        if CONCLUSION_PATTERNS.search(sent):
            selected_indices.add(i)
    selected_indices = sorted(selected_indices)[:25]
    summary = " ".join([sents[i] for i in selected_indices if i < n])
    return limit_tokens(summary, max_tokens)

def legal_sbert_weighted(text, max_tokens=512):
    sents = sent_tokenize(text)
    if not sents: return ""
    n = len(sents)
    weights = torch.ones(n)
    for i, sent in enumerate(sents):
        if CONCLUSION_PATTERNS.search(sent): weights[i] *= 3.0
        if i > n * 0.75: weights[i] *= 2.5
        elif i < n * 0.1: weights[i] *= 1.5
    emb = model.encode(sents, convert_to_tensor=True)
    weighted_centroid = (emb * weights.unsqueeze(1).to(emb.device)).sum(dim=0) / weights.sum()
    weighted_centroid = weighted_centroid.unsqueeze(0)
    sims = cos_sim(weighted_centroid, emb)[0]
    topk = min(20, n)
    idxs = sims.topk(k=topk).indices.tolist()
    idxs.sort()
    summary = " ".join([sents[i] for i in idxs])
    return limit_tokens(summary, max_tokens)

# ---- Methods List ----
methods = ["TextRank", "LexRank", "LSA", "KL-Sum", "SumBasic", "Reduction", "LegalSBERT-Weighted"]
timing_stats = {m: [] for m in methods}

# ---- Process Document ----
def process_document(doc_id, text):
    text = str(text)
    if not text.strip(): return []
    text = preprocess_legal_text(text, max_words=10000)
    
    out = []
    for m in methods:
        start = time.time()
        try:
            if m == "LegalSBERT-Weighted": s = legal_sbert_weighted(text, MAX_TOKENS)
            elif m == "Reduction": s = reduction_summarize(text, MAX_TOKENS)
            else: s = sumy_summarize(text, m, MAX_TOKENS)
        except Exception as e:
            sents = sent_tokenize(text)
            n = len(sents)
            strategic = list(range(min(5, n))) + list(range(max(0, n-10), n))
            s = " ".join([sents[i] for i in strategic if i < n])
            s = limit_tokens(s, MAX_TOKENS)
        
        duration = time.time() - start
        timing_stats[m].append(duration)
        out.append((doc_id, m, limit_tokens(s, MAX_TOKENS)))
    
    return out

# ---- Enhanced Evaluation with Ground Truth ----
def evaluate_with_ground_truth(extracted_df, original_df, text_col, summary_col):
    """Comprehensive evaluation using ground truth summaries"""
    evaluation_results = []
    
    print("🔍 Evaluating with ground truth summaries...")
    
    for doc_id in extracted_df['doc_id'].unique():
        # Get ground truth summary
        if doc_id in original_df.index:
            ground_truth = str(original_df.loc[doc_id, summary_col])
        else:
            continue
            
        doc_summaries = extracted_df[extracted_df['doc_id'] == doc_id]
        original_text = original_df.loc[doc_id, text_col]
        
        for _, row in doc_summaries.iterrows():
            method = row['method']
            predicted_summary = row['extractive_doc']
            
            # Calculate all metrics
            metrics = {
                'doc_id': doc_id,
                'method': method,
                'summary_length': len(predicted_summary.split()),
                'legal_score': validate_legal_summary(predicted_summary),
                'has_conclusion': bool(CONCLUSION_PATTERNS.search(predicted_summary)),
            }
            
            # ROUGE scores
            rouge_scores = calculate_rouge(predicted_summary, ground_truth)
            metrics.update(rouge_scores)
            
            # BERTScore
            bert_scores = calculate_bertscore(predicted_summary, ground_truth)
            metrics.update(bert_scores)
            
            evaluation_results.append(metrics)
    
    eval_df = pd.DataFrame(evaluation_results)
    
    # Print comprehensive results
    print_comprehensive_results(eval_df)
    
    return eval_df

def print_comprehensive_results(eval_df):
    """Print detailed evaluation results"""
    print("\n" + "="*80)
    print("🎯 COMPREHENSIVE EVALUATION WITH GROUND TRUTH SUMMARIES")
    print("="*80)
    
    # Group by method
    summary = eval_df.groupby('method').agg({
        'rouge1_f1': ['mean', 'std'],
        'rouge2_f1': ['mean', 'std'],
        'rougeL_f1': ['mean', 'std'], 
        'bertscore_f1': ['mean', 'std'],
        'has_conclusion': 'mean',
        'legal_score': 'mean',
        'summary_length': 'mean',
        'doc_id': 'count'
    }).round(4)
    
    # Flatten column names
    summary.columns = ['_'.join(col).strip() for col in summary.columns.values]
    print(summary)
    
    # Identify best methods
    print(f"\n🏆 TOP PERFORMING METHODS:")
    print("   By ROUGE-1 F1:")
    best_rouge1 = eval_df.groupby('method')['rouge1_f1'].mean().nlargest(3)
    for method, score in best_rouge1.items():
        print(f"     {method}: {score:.3f}")
    
    print("   By BERTScore F1:")
    best_bertscore = eval_df.groupby('method')['bertscore_f1'].mean().nlargest(3)
    for method, score in best_bertscore.items():
        print(f"     {method}: {score:.3f}")
    
    print("   By Conclusion Inclusion:")
    best_conclusion = eval_df.groupby('method')['has_conclusion'].mean().nlargest(3)
    for method, score in best_conclusion.items():
        print(f"     {method}: {score:.1%}")

# ---- Main Execution ----
if __name__ == "__main__":
    # Setup multi-GPU
    setup_multigpu()
    
    # Load dataset
    df = load_dataset(INPUT_PATH)
    text_col, summary_col = detect_columns(df)
    print(f"📄 Text column: '{text_col}'")
    print(f"📝 Summary column: '{summary_col}'")
    
    # TEST MODE or FULL RUN
    if TEST_MODE:
        df = df.head(TEST_SIZE)
        print(f"🧪 TEST MODE: Processing first {TEST_SIZE} documents")
    else:
        print(f"🚀 FULL RUN: Processing all {len(df)} documents")
    
    # ✅ ENHANCED: Load checkpoint
    processed_ids, checkpoint_timing_stats = load_checkpoint()
    if checkpoint_timing_stats:
        timing_stats.update(checkpoint_timing_stats)
    
    # Process documents
    rows = []
    to_process = [(idx, df.loc[idx, text_col]) for idx in df.index if idx not in processed_ids]
    total_docs = len(to_process)
    start_time = time.time()

    print(f"\n🚀 Processing {total_docs} remaining docs...")
    
    if total_docs > 0:
        for i, (doc_id, text) in enumerate(tqdm(to_process, total=total_docs), start=1):
            result = process_document(doc_id, text)
            rows.extend(result)
            processed_ids.add(doc_id)
            
            # ✅ ENHANCED: Frequent checkpoints
            if i % CHUNK_SIZE == 0:
                # Save current progress
                all_rows = rows  # In full run, we're building from scratch after checkpoint
                extracted_df = pd.DataFrame(all_rows, columns=["doc_id","method","extractive_doc"])
                extracted_df.to_csv(OUTPUT_CSV, index=False)
                
                # Save checkpoint
                save_checkpoint(processed_ids, rows, timing_stats)
                
                elapsed = time.time() - start_time
                docs_per_min = i / (elapsed / 60) if elapsed > 0 else 0
                remaining = (total_docs - i) / docs_per_min if docs_per_min > 0 else 0
                print(f"💾 Checkpoint: {i}/{total_docs} | {docs_per_min:.1f} docs/min | ETA: {remaining:.1f} min")

    # Final save
    extracted_df = pd.DataFrame(rows, columns=["doc_id","method","extractive_doc"])
    extracted_df.to_csv(OUTPUT_CSV, index=False)
    
    # Save final checkpoint (completed)
    save_checkpoint(processed_ids, rows, timing_stats)
    
    # Run comprehensive evaluation with ground truth
    print("\n🔍 Running comprehensive evaluation with ground truth...")
    evaluation_df = evaluate_with_ground_truth(extracted_df, df, text_col, summary_col)
    evaluation_df.to_csv(EVAL_CSV, index=False)
    
    # Final stats
    elapsed_total = time.time() - start_time
    print(f"\n✅ Completed in {elapsed_total/60:.1f} min")
    print(f"📊 Generated {len(rows)} summaries")
    print(f"💾 Output: {OUTPUT_CSV}")
    print(f"📈 Evaluation: {EVAL_CSV}")
    print(f"💾 Checkpoint: {CHECKPOINT_FILE} (delete this file to start fresh)")

In [ ]:
# === COMPLETE EVALUATION WITH DEPENDENCIES ===

# Install dependencies
print("📦 Installing required packages...")
!pip install rouge-score nltk tqdm
import pandas as pd
import nltk
from rouge_score import rouge_scorer
from tqdm.notebook import tqdm
import re

# Download NLTK data
nltk.download('punkt')
print("✅ All dependencies loaded!")

# Initialize ROUGE scorer
rouge_scorer_obj = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True)

# Define your functions
def calculate_rouge(predicted, reference):
    scores = rouge_scorer_obj.score(reference, predicted)
    return {
        'rouge1_f1': scores['rouge1'].fmeasure,
        'rouge2_f1': scores['rouge2'].fmeasure,
        'rougeL_f1': scores['rougeL'].fmeasure,
        'rougeLsum_f1': scores['rougeLsum'].fmeasure,
        'rouge1_precision': scores['rouge1'].precision,
        'rouge1_recall': scores['rouge1'].recall,
    }

def validate_legal_summary(summary):
    legal_indicators = ['court', 'judge', 'appeal', 'case', 'law', 'legal', 
                       'ruling', 'decision', 'therefore', 'thus', 'accordingly',
                       'held', 'conclusion', 'finding']
    summary_lower = summary.lower()
    return sum(1 for indicator in legal_indicators if indicator in summary_lower) >= 3

CONCLUSION_PATTERNS = re.compile(
    r'\b(held|hold that|we hold|we find|concluded?|ruling|final order|'
    r'appeals? (?:allowed|dismissed)|petition (?:allowed|dismissed)|'
    r'consequently|therefore|thus|accordingly|in view of|'
    r'we (?:allow|dismiss|set aside|uphold|reverse|affirm))\b',
    re.IGNORECASE
)

# NOW run your evaluation code...
print("📂 Loading complete merged summaries...")
complete_extracted_df = pd.read_csv('/kaggle/input/legaldoc1/extracted_summaries_with_eval.csv')
# ... [rest of your evaluation code] ...

# === LOAD COMPLETE MERGED DATA ===
print("📂 Loading complete merged summaries...")

# Load the complete merged CSV
complete_extracted_df = pd.read_csv('/kaggle/input/legaldoc1/extracted_summaries_with_eval.csv')

print(f"✅ Loaded complete dataset: {len(complete_extracted_df)} rows")
print(f"📊 Expected: 5532 docs × 7 methods = 38,724 rows")
print(f"📈 Actual: {len(complete_extracted_df)} rows")

# Check methods distribution
methods_count = complete_extracted_df['method'].value_counts()
print(f"\n📋 Methods distribution:")
print(methods_count)

# Check document coverage  
doc_coverage = complete_extracted_df['doc_id'].nunique()
print(f"\n📄 Documents covered: {doc_coverage}/5532 ({(doc_coverage/5532)*100:.1f}%)")

# Now run evaluation on COMPLETE data
print("\n🔍 Running ROUGE evaluation on COMPLETE dataset...")

def evaluate_rouge_only_with_progress(extracted_df, original_df, text_col, summary_col):
    """ROUGE-only evaluation with progress bar"""
    evaluation_results = []
    
    # Get unique document-method pairs for progress tracking
    total_pairs = len(extracted_df)
    print(f"Evaluating {total_pairs} summary-ground_truth pairs...")
    
    # Use tqdm for progress bar
    from tqdm.notebook import tqdm
    
    for idx, (_, row) in enumerate(tqdm(extracted_df.iterrows(), total=total_pairs, desc="Evaluating")):
        doc_id = row['doc_id']
        method = row['method']
        predicted_summary = row['extractive_doc']
        
        # Get ground truth summary
        if doc_id in original_df.index:
            ground_truth = str(original_df.loc[doc_id, summary_col])
        else:
            continue
            
        # Calculate metrics
        metrics = {
            'doc_id': doc_id,
            'method': method,
            'summary_length': len(predicted_summary.split()),
            'legal_score': validate_legal_summary(predicted_summary),
            'has_conclusion': bool(CONCLUSION_PATTERNS.search(predicted_summary)),
        }
        
        # ROUGE scores only (memory efficient)
        try:
            rouge_scores = calculate_rouge(predicted_summary, ground_truth)
            metrics.update(rouge_scores)
        except Exception as e:
            print(f"⚠️ ROUGE failed for doc {doc_id}, method {method}: {e}")
            # Set default scores if ROUGE fails
            metrics.update({
                'rouge1_f1': 0.0, 'rouge2_f1': 0.0, 'rougeL_f1': 0.0,
                'rougeLsum_f1': 0.0, 'rouge1_precision': 0.0, 'rouge1_recall': 0.0
            })
        
        evaluation_results.append(metrics)
        
        # Progress update every 1000 pairs
        if (idx + 1) % 1000 == 0:
            print(f"  Completed {idx + 1}/{total_pairs} pairs...")
            # Clear memory periodically
            import gc
            gc.collect()
    
    return pd.DataFrame(evaluation_results)

# Run evaluation on COMPLETE data
df = pd.read_csv("/kaggle/input/train-data/train_data.csv")
evaluation_df = evaluate_rouge_only_with_progress(complete_extracted_df, df, 'judgment', "summary")

EVAL_CSV = "/kaggle/working/comprehensive_evaluation.csv"

# Save results
evaluation_df.to_csv(EVAL_CSV, index=False)
print(f"✅ Complete evaluation saved to: {EVAL_CSV}")

# Show comprehensive results
print_comprehensive_results(evaluation_df)

print(f"\n🎉 COMPLETE EVALUATION FINISHED!")
print(f"📊 Total summaries evaluated: {len(evaluation_df)}")
print(f"💾 Results saved: {EVAL_CSV}")

In [ ]:
def print_comprehensive_results(eval_df):
    """Print detailed evaluation results"""
    print("\n" + "="*80)
    print("🎯 COMPREHENSIVE EVALUATION WITH GROUND TRUTH SUMMARIES")
    print("="*80)
    
    # Group by method
    summary = eval_df.groupby('method').agg({
        'rouge1_f1': ['mean', 'std'],
        'rouge2_f1': ['mean', 'std'],
        'rougeL_f1': ['mean', 'std'], 
        'rougeLsum_f1':['mean', 'std'],
        'rouge1_precision':['mean', 'std'],
        'rouge1_recall':['mean', 'std'],
        'has_conclusion': 'mean',
        'legal_score': 'mean',
        'summary_length': 'mean',
        'doc_id': 'count'
    }).round(4)
    
    # Flatten column names
    summary.columns = ['_'.join(col).strip() for col in summary.columns.values]
    print(summary)
    
    # Identify best methods
    print(f"\n🏆 TOP PERFORMING METHODS:")
    print("   By ROUGE-1 F1:")
    best_rouge1 = eval_df.groupby('method')['rouge1_f1'].mean().nlargest(3)
    for method, score in best_rouge1.items():
        print(f"     {method}: {score:.3f}")
    
    print("   By Conclusion Inclusion:")
    best_conclusion = eval_df.groupby('method')['has_conclusion'].mean().nlargest(3)
    for method, score in best_conclusion.items():
        print(f"     {method}: {score:.1%}")

# Show comprehensive results
print_comprehensive_results(evaluation_df)

print(f"\n🎉 COMPLETE EVALUATION FINISHED!")
print(f"📊 Total summaries evaluated: {len(evaluation_df)}")
print(f"💾 Results saved: {EVAL_CSV}")

✅ ROUGE installed
✅ BERTScore installed
🔧 Loading: law-ai/InLegalBERT
🚀 Using 2 GPUs with DataParallel
✅ Model loaded!
🖥️  Device: cuda
✓ SBERT ready
🎯 Using 2 GPUs!
✓ Loading CSV: /kaggle/input/train-data/train_data.csv
📊 Dataset shape: (5532, 2)
📋 Columns: ['judgment', 'summary']
📄 Text column: 'judgment'
📝 Summary column: 'summary'
🚀 FULL RUN: Processing all 5532 documents

🚀 Processing 5532 remaining docs...
 95%
 5229/5532 [11:42:35<25:59,  5.15s/it]
💾 Checkpoint saved: 50 docs processed
💾 Checkpoint: 50/5532 | 6.2 docs/min | ETA: 885.7 min
💾 Checkpoint saved: 100 docs processed
💾 Checkpoint: 100/5532 | 6.5 docs/min | ETA: 836.3 min
💾 Checkpoint saved: 150 docs processed
💾 Checkpoint: 150/5532 | 7.1 docs/min | ETA: 753.6 min
💾 Checkpoint saved: 200 docs processed
💾 Checkpoint: 200/5532 | 7.1 docs/min | ETA: 755.2 min
💾 Checkpoint saved: 250 docs processed
💾 Checkpoint: 250/5532 | 7.2 docs/min | ETA: 738.3 min
💾 Checkpoint saved: 300 docs processed
💾 Checkpoint: 300/5532 | 7.3 docs/min | ETA: 720.8 min
💾 Checkpoint saved: 350 docs processed
💾 Checkpoint: 350/5532 | 7.2 docs/min | ETA: 724.5 min
💾 Checkpoint saved: 400 docs processed
💾 Checkpoint: 400/5532 | 7.2 docs/min | ETA: 711.4 min
💾 Checkpoint saved: 450 docs processed
💾 Checkpoint: 450/5532 | 7.6 docs/min | ETA: 669.8 min
💾 Checkpoint saved: 500 docs processed
💾 Checkpoint: 500/5532 | 7.7 docs/min | ETA: 651.5 min
💾 Checkpoint saved: 550 docs processed
💾 Checkpoint: 550/5532 | 7.8 docs/min | ETA: 640.2 min
💾 Checkpoint saved: 600 docs processed
💾 Checkpoint: 600/5532 | 7.6 docs/min | ETA: 649.4 min
💾 Checkpoint saved: 650 docs processed
💾 Checkpoint: 650/5532 | 7.5 docs/min | ETA: 655.2 min
💾 Checkpoint saved: 700 docs processed
💾 Checkpoint: 700/5532 | 7.5 docs/min | ETA: 646.1 min
💾 Checkpoint saved: 750 docs processed
💾 Checkpoint: 750/5532 | 7.3 docs/min | ETA: 650.8 min
💾 Checkpoint saved: 800 docs processed
💾 Checkpoint: 800/5532 | 7.4 docs/min | ETA: 642.1 min
💾 Checkpoint saved: 850 docs processed
💾 Checkpoint: 850/5532 | 7.4 docs/min | ETA: 632.3 min
💾 Checkpoint saved: 900 docs processed
💾 Checkpoint: 900/5532 | 7.3 docs/min | ETA: 630.5 min
💾 Checkpoint saved: 950 docs processed
💾 Checkpoint: 950/5532 | 7.4 docs/min | ETA: 622.7 min
💾 Checkpoint saved: 1000 docs processed
💾 Checkpoint: 1000/5532 | 7.3 docs/min | ETA: 620.3 min
💾 Checkpoint saved: 1050 docs processed
💾 Checkpoint: 1050/5532 | 7.4 docs/min | ETA: 606.5 min
💾 Checkpoint saved: 1100 docs processed
💾 Checkpoint: 1100/5532 | 7.3 docs/min | ETA: 609.2 min
💾 Checkpoint saved: 1150 docs processed
💾 Checkpoint: 1150/5532 | 7.3 docs/min | ETA: 602.5 min
💾 Checkpoint saved: 1200 docs processed
💾 Checkpoint: 1200/5532 | 7.3 docs/min | ETA: 593.6 min
💾 Checkpoint saved: 1250 docs processed
💾 Checkpoint: 1250/5532 | 7.2 docs/min | ETA: 591.8 min
💾 Checkpoint saved: 1300 docs processed
💾 Checkpoint: 1300/5532 | 7.2 docs/min | ETA: 584.4 min
💾 Checkpoint saved: 1350 docs processed
💾 Checkpoint: 1350/5532 | 7.2 docs/min | ETA: 576.9 min
💾 Checkpoint saved: 1400 docs processed
💾 Checkpoint: 1400/5532 | 7.3 docs/min | ETA: 569.9 min
💾 Checkpoint saved: 1450 docs processed
💾 Checkpoint: 1450/5532 | 7.2 docs/min | ETA: 564.7 min
💾 Checkpoint saved: 1500 docs processed
💾 Checkpoint: 1500/5532 | 7.3 docs/min | ETA: 555.9 min
💾 Checkpoint saved: 1550 docs processed
💾 Checkpoint: 1550/5532 | 7.3 docs/min | ETA: 546.1 min
💾 Checkpoint saved: 1600 docs processed
💾 Checkpoint: 1600/5532 | 7.2 docs/min | ETA: 543.8 min
💾 Checkpoint saved: 1650 docs processed
💾 Checkpoint: 1650/5532 | 7.3 docs/min | ETA: 530.6 min
💾 Checkpoint saved: 1700 docs processed
💾 Checkpoint: 1700/5532 | 7.3 docs/min | ETA: 526.6 min
💾 Checkpoint saved: 1750 docs processed
💾 Checkpoint: 1750/5532 | 7.3 docs/min | ETA: 515.7 min
💾 Checkpoint saved: 1800 docs processed
💾 Checkpoint: 1800/5532 | 7.3 docs/min | ETA: 509.3 min
💾 Checkpoint saved: 1850 docs processed
💾 Checkpoint: 1850/5532 | 7.3 docs/min | ETA: 505.2 min
💾 Checkpoint saved: 1900 docs processed
💾 Checkpoint: 1900/5532 | 7.3 docs/min | ETA: 499.4 min
💾 Checkpoint saved: 1950 docs processed
💾 Checkpoint: 1950/5532 | 7.3 docs/min | ETA: 491.5 min
💾 Checkpoint saved: 2000 docs processed
💾 Checkpoint: 2000/5532 | 7.3 docs/min | ETA: 483.4 min
💾 Checkpoint saved: 2050 docs processed
💾 Checkpoint: 2050/5532 | 7.2 docs/min | ETA: 482.4 min
💾 Checkpoint saved: 2100 docs processed
💾 Checkpoint: 2100/5532 | 7.2 docs/min | ETA: 475.6 min
💾 Checkpoint saved: 2150 docs processed
💾 Checkpoint: 2150/5532 | 7.2 docs/min | ETA: 467.6 min
💾 Checkpoint saved: 2200 docs processed
💾 Checkpoint: 2200/5532 | 7.3 docs/min | ETA: 459.5 min
💾 Checkpoint saved: 2250 docs processed
💾 Checkpoint: 2250/5532 | 7.3 docs/min | ETA: 451.8 min
💾 Checkpoint saved: 2300 docs processed
💾 Checkpoint: 2300/5532 | 7.2 docs/min | ETA: 446.8 min
💾 Checkpoint saved: 2350 docs processed
💾 Checkpoint: 2350/5532 | 7.3 docs/min | ETA: 438.3 min
💾 Checkpoint saved: 2400 docs processed
💾 Checkpoint: 2400/5532 | 7.2 docs/min | ETA: 435.4 min
💾 Checkpoint saved: 2450 docs processed
💾 Checkpoint: 2450/5532 | 7.2 docs/min | ETA: 427.5 min
💾 Checkpoint saved: 2500 docs processed
💾 Checkpoint: 2500/5532 | 7.2 docs/min | ETA: 419.1 min
💾 Checkpoint saved: 2550 docs processed
💾 Checkpoint: 2550/5532 | 7.3 docs/min | ETA: 410.5 min
💾 Checkpoint saved: 2600 docs processed
💾 Checkpoint: 2600/5532 | 7.3 docs/min | ETA: 402.5 min
💾 Checkpoint saved: 2650 docs processed
💾 Checkpoint: 2650/5532 | 7.3 docs/min | ETA: 393.6 min
💾 Checkpoint saved: 2700 docs processed
💾 Checkpoint: 2700/5532 | 7.3 docs/min | ETA: 386.8 min
💾 Checkpoint saved: 2750 docs processed
💾 Checkpoint: 2750/5532 | 7.3 docs/min | ETA: 380.1 min
💾 Checkpoint saved: 2800 docs processed
💾 Checkpoint: 2800/5532 | 7.3 docs/min | ETA: 373.0 min
💾 Checkpoint saved: 2850 docs processed
💾 Checkpoint: 2850/5532 | 7.3 docs/min | ETA: 366.7 min
💾 Checkpoint saved: 2900 docs processed
💾 Checkpoint: 2900/5532 | 7.4 docs/min | ETA: 358.0 min
💾 Checkpoint saved: 2950 docs processed
💾 Checkpoint: 2950/5532 | 7.3 docs/min | ETA: 351.7 min
💾 Checkpoint saved: 3000 docs processed
💾 Checkpoint: 3000/5532 | 7.4 docs/min | ETA: 343.6 min
💾 Checkpoint saved: 3050 docs processed
💾 Checkpoint: 3050/5532 | 7.4 docs/min | ETA: 337.1 min
💾 Checkpoint saved: 3100 docs processed
💾 Checkpoint: 3100/5532 | 7.4 docs/min | ETA: 329.1 min
💾 Checkpoint saved: 3150 docs processed
💾 Checkpoint: 3150/5532 | 7.4 docs/min | ETA: 323.5 min
💾 Checkpoint saved: 3200 docs processed
💾 Checkpoint: 3200/5532 | 7.3 docs/min | ETA: 317.6 min
💾 Checkpoint saved: 3250 docs processed
💾 Checkpoint: 3250/5532 | 7.3 docs/min | ETA: 311.9 min
💾 Checkpoint saved: 3300 docs processed
💾 Checkpoint: 3300/5532 | 7.3 docs/min | ETA: 306.9 min
💾 Checkpoint saved: 3350 docs processed
💾 Checkpoint: 3350/5532 | 7.3 docs/min | ETA: 300.8 min
💾 Checkpoint saved: 3400 docs processed
💾 Checkpoint: 3400/5532 | 7.3 docs/min | ETA: 294.0 min
💾 Checkpoint saved: 3450 docs processed
💾 Checkpoint: 3450/5532 | 7.3 docs/min | ETA: 285.4 min
💾 Checkpoint saved: 3500 docs processed
💾 Checkpoint: 3500/5532 | 7.3 docs/min | ETA: 279.0 min
💾 Checkpoint saved: 3550 docs processed
💾 Checkpoint: 3550/5532 | 7.3 docs/min | ETA: 271.5 min
💾 Checkpoint saved: 3600 docs processed
💾 Checkpoint: 3600/5532 | 7.3 docs/min | ETA: 264.7 min
💾 Checkpoint saved: 3650 docs processed
💾 Checkpoint: 3650/5532 | 7.3 docs/min | ETA: 257.7 min
💾 Checkpoint saved: 3700 docs processed
💾 Checkpoint: 3700/5532 | 7.3 docs/min | ETA: 250.0 min
💾 Checkpoint saved: 3750 docs processed
💾 Checkpoint: 3750/5532 | 7.3 docs/min | ETA: 243.9 min
💾 Checkpoint saved: 3800 docs processed
💾 Checkpoint: 3800/5532 | 7.3 docs/min | ETA: 236.4 min
💾 Checkpoint saved: 3850 docs processed
💾 Checkpoint: 3850/5532 | 7.3 docs/min | ETA: 229.5 min
💾 Checkpoint saved: 3900 docs processed
💾 Checkpoint: 3900/5532 | 7.3 docs/min | ETA: 222.3 min
💾 Checkpoint saved: 3950 docs processed
💾 Checkpoint: 3950/5532 | 7.3 docs/min | ETA: 216.0 min
💾 Checkpoint saved: 4000 docs processed
💾 Checkpoint: 4000/5532 | 7.3 docs/min | ETA: 208.5 min
💾 Checkpoint saved: 4050 docs processed
💾 Checkpoint: 4050/5532 | 7.3 docs/min | ETA: 201.9 min
💾 Checkpoint saved: 4100 docs processed
💾 Checkpoint: 4100/5532 | 7.4 docs/min | ETA: 194.5 min
💾 Checkpoint saved: 4150 docs processed
💾 Checkpoint: 4150/5532 | 7.4 docs/min | ETA: 186.8 min
💾 Checkpoint saved: 4200 docs processed
💾 Checkpoint: 4200/5532 | 7.4 docs/min | ETA: 180.3 min
💾 Checkpoint saved: 4250 docs processed
💾 Checkpoint: 4250/5532 | 7.4 docs/min | ETA: 173.4 min
💾 Checkpoint saved: 4300 docs processed
💾 Checkpoint: 4300/5532 | 7.4 docs/min | ETA: 166.2 min
💾 Checkpoint saved: 4350 docs processed
💾 Checkpoint: 4350/5532 | 7.4 docs/min | ETA: 159.1 min
💾 Checkpoint saved: 4400 docs processed
💾 Checkpoint: 4400/5532 | 7.4 docs/min | ETA: 152.2 min
💾 Checkpoint saved: 4450 docs processed
💾 Checkpoint: 4450/5532 | 7.5 docs/min | ETA: 145.2 min
💾 Checkpoint saved: 4500 docs processed
💾 Checkpoint: 4500/5532 | 7.4 docs/min | ETA: 138.9 min
💾 Checkpoint saved: 4550 docs processed
💾 Checkpoint: 4550/5532 | 7.4 docs/min | ETA: 132.1 min
💾 Checkpoint saved: 4600 docs processed
💾 Checkpoint: 4600/5532 | 7.4 docs/min | ETA: 125.2 min
💾 Checkpoint saved: 4650 docs processed
💾 Checkpoint: 4650/5532 | 7.4 docs/min | ETA: 118.6 min
💾 Checkpoint saved: 4700 docs processed
💾 Checkpoint: 4700/5532 | 7.4 docs/min | ETA: 111.9 min
💾 Checkpoint saved: 4750 docs processed
💾 Checkpoint: 4750/5532 | 7.4 docs/min | ETA: 105.4 min
💾 Checkpoint saved: 4800 docs processed
💾 Checkpoint: 4800/5532 | 7.4 docs/min | ETA: 98.8 min
💾 Checkpoint saved: 4850 docs processed
💾 Checkpoint: 4850/5532 | 7.4 docs/min | ETA: 91.8 min
💾 Checkpoint saved: 4900 docs processed
💾 Checkpoint: 4900/5532 | 7.4 docs/min | ETA: 84.9 min
💾 Checkpoint saved: 4950 docs processed
💾 Checkpoint: 4950/5532 | 7.4 docs/min | ETA: 78.4 min
💾 Checkpoint saved: 5000 docs processed
💾 Checkpoint: 5000/5532 | 7.4 docs/min | ETA: 71.6 min
💾 Checkpoint saved: 5050 docs processed
💾 Checkpoint: 5050/5532 | 7.4 docs/min | ETA: 64.9 min
💾 Checkpoint saved: 5100 docs processed
💾 Checkpoint: 5100/5532 | 7.4 docs/min | ETA: 58.2 min
💾 Checkpoint saved: 5150 docs processed
💾 Checkpoint: 5150/5532 | 7.4 docs/min | ETA: 51.5 min
💾 Checkpoint saved: 5200 docs processed
💾 Checkpoint: 5200/5532 | 7.4 docs/min | ETA: 44.7 min


🔧 Loading: law-ai/InLegalBERT

🚀 Using 2 GPUs with DataParallel
✅ Model loaded!
🖥️  Device: cuda
✓ SBERT ready
🎯 Using 2 GPUs!
✓ Loading CSV: /kaggle/input/train-data/train_data.csv
📊 Dataset shape: (5532, 2)
📋 Columns: ['judgment', 'summary']
📄 Text column: 'judgment'
📝 Summary column: 'summary'
🚀 FULL RUN: Processing all 5532 documents
📂 Loading checkpoint...
✓ Resuming from checkpoint: 5200 docs already processed

🚀 Processing 332 remaining docs...
100%
 332/332 [35:51<00:00,  8.32s/it]

💾 Checkpoint saved: 5250 docs processed
💾 Checkpoint: 50/332 | 11.1 docs/min | ETA: 25.4 min
💾 Checkpoint saved: 5300 docs processed
💾 Checkpoint: 100/332 | 9.8 docs/min | ETA: 23.6 min
💾 Checkpoint saved: 5350 docs processed
💾 Checkpoint: 150/332 | 9.7 docs/min | ETA: 18.8 min
💾 Checkpoint saved: 5400 docs processed
💾 Checkpoint: 200/332 | 10.2 docs/min | ETA: 13.0 min
💾 Checkpoint saved: 5450 docs processed
💾 Checkpoint: 250/332 | 9.8 docs/min | ETA: 8.4 min
💾 Checkpoint saved: 5500 docs processed
💾 Checkpoint: 300/332 | 9.6 docs/min | ETA: 3.3 min
💾 Checkpoint saved: 5532 docs processed